## load libraries

In [1]:
import pandas as pd
import geopandas as gpd
import folium
import rasterio

from shapely.geometry import Polygon, Point
import numpy as np
import xarray as xr
import random
import os
import time
import re
import math

import matplotlib.pyplot as plt
import altair as alt

In [2]:
# Set GeoPandas to use pyogrio
gpd.options.io_engine = "pyogrio"

In [3]:
gpd.show_versions()


SYSTEM INFO
-----------
python     : 3.12.4 | packaged by conda-forge | (main, Jun 17 2024, 10:13:44) [Clang 16.0.6 ]
executable : /Users/jwhite/miniforge3/envs/siads699b/bin/python
machine    : macOS-14.5-arm64-arm-64bit

GEOS, GDAL, PROJ INFO
---------------------
GEOS       : 3.12.2
GEOS lib   : None
GDAL       : 3.9.1
GDAL data dir: /Users/jwhite/miniforge3/envs/siads699b/share/gdal/
PROJ       : 9.4.0
PROJ data dir: /Users/jwhite/miniforge3/envs/siads699b/share/proj

PYTHON DEPENDENCIES
-------------------
geopandas  : 1.0.1
numpy      : 2.0.0
pandas     : 2.2.2
pyproj     : 3.6.1
shapely    : 2.0.5
pyogrio    : 0.9.0
geoalchemy2: 0.15.2
geopy      : 2.4.1
matplotlib : 3.9.1
mapclassify: 2.6.1
fiona      : 1.9.6
psycopg    : 3.2.1
psycopg2   : 2.9.9 (dt dec pq3 ext lo64)
pyarrow    : 16.1.0


In [5]:
csb_crop_codes_and_colors = pd.read_csv('../data/fields/crop_codes_and_colors.csv')

In [6]:
csb_crop_codes_and_colors

,Codes,Class_Names,Crop (1) / non-crop (0),ESRI_Red,ESRI_Green,ESRI_Blue
0,0,Background,0,0,0,0
1,1,Corn,1,255,212,0
2,2,Cotton,1,255,38,38
3,3,Rice,1,0,169,230
4,4,Sorghum,1,255,158,15
...,...,...,...,...,...,...
251,251,,0,255,212,0
252,252,,0,38,115,0
253,253,,0,168,112,0
254,254,Dbl Crop Barley/Soybeans,1,38,115,0


In [7]:
csb_crops = csb_crop_codes_and_colors[csb_crop_codes_and_colors['Crop (1) / non-crop (0)']==1]
cdl_codes = csb_crops['Class_Names'].to_dict()
cdl_codes

{1: 'Corn',
 2: 'Cotton',
 3: 'Rice',
 4: 'Sorghum',
 5: 'Soybeans',
 6: 'Sunflower',
 10: 'Peanuts',
 11: 'Tobacco',
 12: 'Sweet Corn',
 13: 'Pop or Orn Corn',
 14: 'Mint',
 21: 'Barley',
 22: 'Durum Wheat',
 23: 'Spring Wheat',
 24: 'Winter Wheat',
 25: 'Other Small Grains',
 26: 'Dbl Crop WinWht/Soybeans',
 27: 'Rye',
 28: 'Oats',
 29: 'Millet',
 30: 'Speltz',
 31: 'Canola',
 32: 'Flaxseed',
 33: 'Safflower',
 34: 'Rape Seed',
 35: 'Mustard',
 36: 'Alfalfa',
 37: 'Other Hay/Non Alfalfa',
 38: 'Camelina',
 39: 'Buckwheat',
 41: 'Sugarbeets',
 42: 'Dry Beans',
 43: 'Potatoes',
 44: 'Other Crops',
 45: 'Sugarcane',
 46: 'Sweet Potatoes',
 47: 'Misc Vegs & Fruits',
 48: 'Watermelons',
 49: 'Onions',
 50: 'Cucumbers',
 51: 'Chick Peas',
 52: 'Lentils',
 53: 'Peas',
 54: 'Tomatoes',
 55: 'Caneberries',
 56: 'Hops',
 57: 'Herbs',
 66: 'Cherries',
 67: 'Peaches',
 68: 'Apples',
 69: 'Grapes',
 70: 'Christmas Trees',
 71: 'Other Tree Crops',
 72: 'Citrus',
 74: 'Pecans',
 75: 'Almonds',
 76:

In [8]:
csb_crops['Codes'].unique()

array([  1,   2,   3,   4,   5,   6,  10,  11,  12,  13,  14,  21,  22,
        23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,
        36,  37,  38,  39,  41,  42,  43,  44,  45,  46,  47,  48,  49,
        50,  51,  52,  53,  54,  55,  56,  57,  66,  67,  68,  69,  70,
        71,  72,  74,  75,  76,  77, 204, 205, 206, 207, 208, 209, 210,
       211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223,
       224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236,
       237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249,
       250, 254])

In [9]:
cdl_codes_species = {
    1: {
        "Crop_Name": 'Corn',
        "Likely_Crop_Species": ['Zea mays'],
        "Discussion": "When a field has 'corn' growing in it, the species is most likely Zea mays. Corn is a staple crop grown worldwide for its versatility and nutritional value."
    },
    2: {
        "Crop_Name": 'Cotton',
        "Likely_Crop_Species": ['Gossypium hirsutum', 'Gossypium barbadense', 'Gossypium arboreum', 'Gossypium herbaceum'],
        "Discussion": "When a field has 'cotton' growing in it, the species is most likely Gossypium hirsutum, which is commonly referred to as upland cotton. This species accounts for approximately 90% of the world's cotton production. Other species of cotton include:\n\n1. Gossypium barbadense (also known as extra-long staple cotton, Pima cotton, or Egyptian cotton)\n2. Gossypium arboreum (tree cotton)\n3. Gossypium herbaceum (Levant cotton)\n\nHowever, Gossypium hirsutum is the predominant species used in agriculture."
    },
    3: {
        "Crop_Name": 'Rice',
        "Likely_Crop_Species": ['Oryza sativa', 'Oryza glaberrima'],
        "Discussion": "When a field has 'rice' growing in it, the species is most likely Oryza sativa, which is the most widely grown and consumed type of rice worldwide. Another species is Oryza glaberrima, also known as African rice, which is cultivated in parts of West Africa."
    },
    4: {
        "Crop_Name": 'Sorghum',
        "Likely_Crop_Species": ['Sorghum bicolor'],
        "Discussion": "When a field has 'sorghum' growing in it, the species is most likely Sorghum bicolor. This species is widely grown for grain, forage, and as a biofuel crop. Sorghum is known for its drought tolerance and versatility in different climates."
    },
    5: {
        "Crop_Name": 'Soybeans',
        "Likely_Crop_Species": ['Glycine max'],
        "Discussion": "When a field has 'soybeans' growing in it, the species is most likely Glycine max. Soybeans are a major source of protein and oil and are used in a variety of food and industrial products."
    },
    6: {
        "Crop_Name": 'Sunflower',
        "Likely_Crop_Species": ['Helianthus annuus'],
        "Discussion": "When a field has 'sunflower' growing in it, the species is most likely Helianthus annuus. Sunflowers are grown for their seeds, which are used for oil production, bird feed, and snacks."
    },
    10: {
        "Crop_Name": 'Peanuts',
        "Likely_Crop_Species": ['Arachis hypogaea'],
        "Discussion": "When a field has 'peanuts' growing in it, the species is most likely Arachis hypogaea. Peanuts are grown for their edible seeds and are a major source of oil and protein."
    },
    11: {
        "Crop_Name": 'Tobacco',
        "Likely_Crop_Species": ['Nicotiana tabacum'],
        "Discussion": "When a field has 'tobacco' growing in it, the species is most likely Nicotiana tabacum. Tobacco is cultivated for its leaves, which are used in the production of cigarettes, cigars, and other tobacco products."
    },
    12: {
        "Crop_Name": 'Sweet Corn',
        "Likely_Crop_Species": ['Zea mays convar. saccharata'],
        "Discussion": "When a field has 'sweet corn' growing in it, the species is most likely Zea mays convar. saccharata. Sweet corn is a variety of maize with a high sugar content, commonly consumed as a vegetable."
    },
    13: {
        "Crop_Name": 'Pop or Orn Corn',
        "Likely_Crop_Species": ['Zea mays'],
        "Discussion": "When a field has 'Pop or Orn Corn' growing in it, the species is also Zea mays, but it refers specifically to varieties that are used for popping or ornamental purposes. These varieties have different kernel properties compared to standard field corn or sweet corn. Popcorn, for example, has a hard outer shell and a starchy interior that expands and pops when heated. Ornamental corn varieties are often grown for their colorful and decorative kernels."
    },
    14: {
        "Crop_Name": 'Mint',
        "Likely_Crop_Species": ['Mentha spicata', 'Mentha piperita'],
        "Discussion": "When a field has 'mint' growing in it, the species is most likely Mentha spicata (spearmint) or Mentha piperita (peppermint). These species are widely cultivated for their aromatic leaves, which are used in culinary, medicinal, and cosmetic products."
    },
    21: {
        "Crop_Name": 'Barley',
        "Likely_Crop_Species": ['Hordeum vulgare'],
        "Discussion": "When a field has 'barley' growing in it, the species is most likely Hordeum vulgare. Barley is a versatile cereal grain used for food, animal feed, and brewing."
    },
    22: {
        "Crop_Name": 'Durum Wheat',
        "Likely_Crop_Species": ['Triticum durum'],
        "Discussion": "When a field has 'durum wheat' growing in it, the species is most likely Triticum durum. Durum wheat is a hard variety of wheat used to make pasta, couscous, and some types of bread."
    },
    23: {
        "Crop_Name": 'Spring Wheat',
        "Likely_Crop_Species": ['Triticum aestivum'],
        "Discussion": "When a field has 'spring wheat' growing in it, the species is most likely Triticum aestivum. Spring wheat is planted in the spring and harvested in late summer, primarily used for bread and other baked goods."
    },
    24: {
        "Crop_Name": 'Winter Wheat',
        "Likely_Crop_Species": ['Triticum aestivum'],
        "Discussion": "When a field has 'winter wheat' growing in it, the species is most likely Triticum aestivum. Winter wheat is planted in the fall, goes dormant over the winter, and is harvested in the spring or early summer. It is used for bread, pasta, and other wheat products."
    },
    25: {
        "Crop_Name": 'Other Small Grains',
        "Likely_Crop_Species": ['Avena sativa', 'Secale cereale', 'Triticosecale'],
        "Discussion": "When a field has 'other small grains' growing in it, it includes a variety of species such as oats (Avena sativa), rye (Secale cereale), and triticale (× Triticosecale). These grains are used for food, animal feed, and other purposes."
    },
    26: {
        "Crop_Name": 'Dbl Crop WinWht/Soybeans',
        "Likely_Crop_Species": ['Triticum aestivum', 'Glycine max'],
        "Discussion": "When a field has 'Dbl Crop WinWht/Soybeans' growing in it, the species are most likely Triticum aestivum (winter wheat) and Glycine max (soybeans). Double cropping involves growing two different crops in the same field in one year, usually winter wheat followed by soybeans."
    },
    27: {
        "Crop_Name": 'Rye',
        "Likely_Crop_Species": ['Secale cereale'],
        "Discussion": "When a field has 'rye' growing in it, the species is most likely Secale cereale. Rye is a cereal grain used for food, animal feed, and as a cover crop."
    },
    28: {
        "Crop_Name": 'Oats',
        "Likely_Crop_Species": ['Avena sativa'],
        "Discussion": "When a field has 'oats' growing in it, the species is most likely Avena sativa. Oats are grown for their edible seeds, which are used in cereals, animal feed, and various food products."
    },
    29: {
        "Crop_Name": 'Millet',
        "Likely_Crop_Species": ['Panicum miliaceum', 'Setaria italica'],
        "Discussion": "When a field has 'millet' growing in it, the species are most likely Panicum miliaceum (proso millet) or Setaria italica (foxtail millet). Millets are small-seeded grasses grown for food and animal feed, especially in arid and semi-arid regions."
    },
    30: {
        "Crop_Name": 'Speltz',
        "Likely_Crop_Species": ['Triticum spelta'],
        "Discussion": "When a field has 'speltz' growing in it, the species is most likely Triticum spelta. Spelt is an ancient grain, a subspecies of wheat, known for its nutty flavor and nutritional benefits."
    },
    31: {
        "Crop_Name": 'Canola',
        "Likely_Crop_Species": ['Brassica napus', 'Brassica rapa'],
        "Discussion": "When a field has 'canola' growing in it, the species are most likely Brassica napus or Brassica rapa. Canola is grown for its seeds, which are processed into canola oil and meal."
    },
    32: {
        "Crop_Name": 'Flaxseed',
        "Likely_Crop_Species": ['Linum usitatissimum'],
        "Discussion": "When a field has 'flaxseed' growing in it, the species is most likely Linum usitatissimum. Flaxseed is cultivated for its seeds, which are used to produce flax oil and as a nutritional supplement."
    },
    33: {
        "Crop_Name": 'Safflower',
        "Likely_Crop_Species": ['Carthamus tinctorius'],
        "Discussion": "When a field has 'safflower' growing in it, the species is most likely Carthamus tinctorius. Safflower is grown for its oil-rich seeds and as a birdseed."
    },
    34: {
        "Crop_Name": 'Rape Seed',
        "Likely_Crop_Species": ['Brassica napus'],
        "Discussion": "When a field has 'rape seed' growing in it, the species is most likely Brassica napus. Rape seed is cultivated for its oil-rich seeds, used to produce rapeseed oil and meal."
    },
    35: {
        "Crop_Name": 'Mustard',
        "Likely_Crop_Species": ['Brassica juncea', 'Brassica nigra', 'Brassica alba'],
        "Discussion": "When a field has 'mustard' growing in it, the species are most likely Brassica juncea (brown mustard), Brassica nigra (black mustard), or Brassica alba (white mustard). Mustard is grown for its seeds, used as a spice and in the production of mustard condiments."
    },
    36: {
        "Crop_Name": 'Alfalfa',
        "Likely_Crop_Species": ['Medicago sativa'],
        "Discussion": "When a field has 'alfalfa' growing in it, the species is most likely Medicago sativa. Alfalfa is a perennial forage crop known for its high nutritional value and use as animal feed."
    },
    37: {
        "Crop_Name": 'Other Hay/Non Alfalfa',
        "Likely_Crop_Species": ['Various species'],
        "Discussion": "When a field has 'other hay/non alfalfa' growing in it, it includes a variety of forage species such as clover (Trifolium spp.), timothy (Phleum pratense), and orchardgrass (Dactylis glomerata). These forages are used for animal feed."
    },
    38: {
        "Crop_Name": 'Camelina',
        "Likely_Crop_Species": ['Camelina sativa'],
        "Discussion": "When a field has 'camelina' growing in it, the species is most likely Camelina sativa. Camelina, also known as false flax, is grown for its oil-rich seeds used in biofuel production and as a nutritional supplement."
    },
    39: {
        "Crop_Name": 'Buckwheat',
        "Likely_Crop_Species": ['Fagopyrum esculentum'],
        "Discussion": "When a field has 'buckwheat' growing in it, the species is most likely Fagopyrum esculentum. Buckwheat is a fast-growing crop cultivated for its seeds, used as a grain substitute and in various food products."
    },
    41: {
        "Crop_Name": 'Sugarbeets',
        "Likely_Crop_Species": ['Beta vulgaris'],
        "Discussion": "When a field has 'sugarbeets' growing in it, the species is most likely Beta vulgaris. Sugarbeets are grown for their high sugar content, used in the production of sugar and various food products."
    },
    42: {
        "Crop_Name": 'Dry Beans',
        "Likely_Crop_Species": ['Phaseolus vulgaris', 'Vigna unguiculata'],
        "Discussion": "When a field has 'dry beans' growing in it, the species are most likely Phaseolus vulgaris (common beans) or Vigna unguiculata (cowpeas). These beans are grown for their edible seeds, which are dried and used in various culinary dishes."
    },
    43: {
        "Crop_Name": 'Potatoes',
        "Likely_Crop_Species": ['Solanum tuberosum'],
        "Discussion": "When a field has 'potatoes' growing in it, the species is most likely Solanum tuberosum. Potatoes are a staple crop grown for their edible tubers, used in a variety of culinary dishes."
    },
    44: {
        "Crop_Name": 'Other Crops',
        "Likely_Crop_Species": ['Various species'],
        "Discussion": "When a field has 'other crops' growing in it, it includes a wide range of species cultivated for various purposes, including vegetables, fruits, and specialty crops."
    },
    45: {
        "Crop_Name": 'Sugarcane',
        "Likely_Crop_Species": ['Saccharum officinarum'],
        "Discussion": "When a field has 'sugarcane' growing in it, the species is most likely Saccharum officinarum. Sugarcane is grown for its high sugar content, used in the production of sugar, ethanol, and other products."
    },
    46: {
        "Crop_Name": 'Sweet Potatoes',
        "Likely_Crop_Species": ['Ipomoea batatas'],
        "Discussion": "When a field has 'sweet potatoes' growing in it, the species is most likely Ipomoea batatas. Sweet potatoes are cultivated for their edible tubers, used in various culinary dishes."
    },
    47: {
        "Crop_Name": 'Misc Vegs & Fruits',
        "Likely_Crop_Species": ['Various species'],
        "Discussion": "When a field has 'misc vegs & fruits' growing in it, it includes a variety of vegetable and fruit species cultivated for food production."
    },
    48: {
        "Crop_Name": 'Watermelons',
        "Likely_Crop_Species": ['Citrullus lanatus'],
        "Discussion": "When a field has 'watermelons' growing in it, the species is most likely Citrullus lanatus. Watermelons are cultivated for their large, juicy fruits, which are enjoyed as a refreshing treat."
    },
    49: {
        "Crop_Name": 'Onions',
        "Likely_Crop_Species": ['Allium cepa'],
        "Discussion": "When a field has 'onions' growing in it, the species is most likely Allium cepa. Onions are a widely cultivated vegetable used in culinary dishes around the world."
    },
    50: {
        "Crop_Name": 'Cucumbers',
        "Likely_Crop_Species": ['Cucumis sativus'],
        "Discussion": "When a field has 'cucumbers' growing in it, the species is most likely Cucumis sativus. Cucumbers are grown for their edible fruits, used fresh in salads or pickled."
    },
        51: {
        "Crop_Name": 'Chick Peas',
        "Likely_Crop_Species": ['Cicer arietinum'],
        "Discussion": "When a field has 'chick peas' growing in it, the species is most likely Cicer arietinum. Chickpeas are cultivated for their edible seeds, which are used in a variety of culinary dishes, including hummus and soups."
    },
    52: {
        "Crop_Name": 'Lentils',
        "Likely_Crop_Species": ['Lens culinaris'],
        "Discussion": "When a field has 'lentils' growing in it, the species is most likely Lens culinaris. Lentils are grown for their edible seeds, which are used in soups, stews, and other culinary dishes."
    },
    53: {
        "Crop_Name": 'Peas',
        "Likely_Crop_Species": ['Pisum sativum'],
        "Discussion": "When a field has 'peas' growing in it, the species is most likely Pisum sativum. Peas are grown for their edible seeds, which can be eaten fresh, frozen, or dried."
    },
    54: {
        "Crop_Name": 'Tomatoes',
        "Likely_Crop_Species": ['Solanum lycopersicum'],
        "Discussion": "When a field has 'tomatoes' growing in it, the species is most likely Solanum lycopersicum. Tomatoes are widely cultivated for their edible fruits, used in a variety of culinary dishes."
    },
    55: {
        "Crop_Name": 'Caneberries',
        "Likely_Crop_Species": ['Rubus spp.'],
        "Discussion": "When a field has 'caneberries' growing in it, the species are most likely various Rubus species, including blackberries and raspberries. Caneberries are grown for their edible fruits, enjoyed fresh or used in jams and desserts."
    },
    56: {
        "Crop_Name": 'Hops',
        "Likely_Crop_Species": ['Humulus lupulus'],
        "Discussion": "When a field has 'hops' growing in it, the species is most likely Humulus lupulus. Hops are cultivated for their flowers, which are used as a bittering, flavoring, and stability agent in beer production."
    },
    57: {
        "Crop_Name": 'Herbs',
        "Likely_Crop_Species": ['Various species'],
        "Discussion": "When a field has 'herbs' growing in it, it includes a variety of species such as basil (Ocimum basilicum), parsley (Petroselinum crispum), and thyme (Thymus vulgaris). Herbs are grown for their aromatic leaves, used in culinary, medicinal, and cosmetic products."
    },
    66: {
        "Crop_Name": 'Cherries',
        "Likely_Crop_Species": ['Prunus avium', 'Prunus cerasus'],
        "Discussion": "When a field has 'cherries' growing in it, the species are most likely Prunus avium (sweet cherries) or Prunus cerasus (sour cherries). Cherries are grown for their edible fruits, enjoyed fresh or used in various culinary dishes."
    },
    67: {
        "Crop_Name": 'Peaches',
        "Likely_Crop_Species": ['Prunus persica'],
        "Discussion": "When a field has 'peaches' growing in it, the species is most likely Prunus persica. Peaches are cultivated for their juicy, sweet fruits, enjoyed fresh or used in various culinary dishes."
    },
    68: {
        "Crop_Name": 'Apples',
        "Likely_Crop_Species": ['Malus domestica'],
        "Discussion": "When a field has 'apples' growing in it, the species is most likely Malus domestica. Apples are one of the most widely cultivated tree fruits, enjoyed fresh or used in various culinary dishes and beverages."
    },
    69: {
        "Crop_Name": 'Grapes',
        "Likely_Crop_Species": ['Vitis vinifera'],
        "Discussion": "When a field has 'grapes' growing in it, the species is most likely Vitis vinifera. Grapes are cultivated for their edible fruits, used fresh, dried (as raisins), or in winemaking."
    },
    70: {
        "Crop_Name": 'Christmas Trees',
        "Likely_Crop_Species": ['Various species'],
        "Discussion": "When a field has 'christmas trees' growing in it, it includes a variety of coniferous tree species such as fir (Abies spp.), spruce (Picea spp.), and pine (Pinus spp.). These trees are grown for use as Christmas decorations."
    },
    71: {
        "Crop_Name": 'Other Tree Crops',
        "Likely_Crop_Species": ['Various species'],
        "Discussion": "When a field has 'other tree crops' growing in it, it includes a variety of tree species cultivated for their fruits, nuts, or other products."
    },
    72: {
        "Crop_Name": 'Citrus',
        "Likely_Crop_Species": ['Citrus sinensis', 'Citrus limon', 'Citrus reticulata', 'Citrus paradisi'],
        "Discussion": "When a field has 'citrus' growing in it, the species are most likely Citrus sinensis (sweet oranges), Citrus limon (lemons), Citrus reticulata (mandarins), or Citrus paradisi (grapefruits). Citrus fruits are enjoyed fresh or used in various culinary and beverage products."
    },
    74: {
        "Crop_Name": 'Pecans',
        "Likely_Crop_Species": ['Carya illinoinensis'],
        "Discussion": "When a field has 'pecans' growing in it, the species is most likely Carya illinoinensis. Pecans are cultivated for their edible nuts, enjoyed fresh or used in various culinary dishes and confections."
    },
    75: {
        "Crop_Name": 'Almonds',
        "Likely_Crop_Species": ['Prunus dulcis'],
        "Discussion": "When a field has 'almonds' growing in it, the species is most likely Prunus dulcis. Almonds are grown for their edible seeds, enjoyed fresh or used in various culinary dishes and confections."
    },
    76: {
        "Crop_Name": 'Walnuts',
        "Likely_Crop_Species": ['Juglans regia'],
        "Discussion": "When a field has 'walnuts' growing in it, the species is most likely Juglans regia. Walnuts are cultivated for their edible nuts, enjoyed fresh or used in various culinary dishes and confections."
    },
    77: {
        "Crop_Name": 'Pears',
        "Likely_Crop_Species": ['Pyrus communis'],
        "Discussion": "When a field has 'pears' growing in it, the species is most likely Pyrus communis. Pears are cultivated for their sweet, juicy fruits, enjoyed fresh or used in various culinary dishes."
    },
    204: {
        "Crop_Name": 'Pistachios',
        "Likely_Crop_Species": ['Pistacia vera'],
        "Discussion": "When a field has 'pistachios' growing in it, the species is most likely Pistacia vera. Pistachios are grown for their edible nuts, enjoyed fresh or used in various culinary dishes and confections."
    },
    205: {
        "Crop_Name": 'Triticale',
        "Likely_Crop_Species": ['× Triticosecale'],
        "Discussion": "When a field has 'triticale' growing in it, the species is most likely × Triticosecale. Triticale is a hybrid of wheat (Triticum) and rye (Secale), grown for its grain and forage."
    },
    206: {
        "Crop_Name": 'Carrots',
        "Likely_Crop_Species": ['Daucus carota'],
        "Discussion": "When a field has 'carrots' growing in it, the species is most likely Daucus carota. Carrots are grown for their edible taproots, used in a variety of culinary dishes."
    },
    207: {
        "Crop_Name": 'Asparagus',
        "Likely_Crop_Species": ['Asparagus officinalis'],
        "Discussion": "When a field has 'asparagus' growing in it, the species is most likely Asparagus officinalis. Asparagus is cultivated for its edible shoots, enjoyed fresh or used in various culinary dishes."
    },
    208: {
        "Crop_Name": 'Garlic',
        "Likely_Crop_Species": ['Allium sativum'],
        "Discussion": "When a field has 'garlic' growing in it, the species is most likely Allium sativum. Garlic is cultivated for its edible bulbs, used as a flavoring in various culinary dishes."
    },
    209: {
        "Crop_Name": 'Cantaloupes',
        "Likely_Crop_Species": ['Cucumis melo'],
        "Discussion": "When a field has 'cantaloupes' growing in it, the species is most likely Cucumis melo. Cantaloupes are cultivated for their sweet, fragrant fruits, enjoyed fresh or used in various culinary dishes."
    },
    210: {
        "Crop_Name": 'Prunes',
        "Likely_Crop_Species": ['Prunus domestica'],
        "Discussion": "When a field has 'prunes' growing in it, the species is most likely Prunus domestica. Prunes are dried plums, enjoyed as a snack or used in various culinary dishes."
    },
    211: {
        "Crop_Name": 'Olives',
        "Likely_Crop_Species": ['Olea europaea'],
        "Discussion": "When a field has 'olives' growing in it, the species is most likely Olea europaea. Olives are cultivated for their edible fruits, enjoyed fresh, cured, or used to produce olive oil."
    },
    212: {
        "Crop_Name": 'Oranges',
        "Likely_Crop_Species": ['Citrus sinensis'],
        "Discussion": "When a field has 'oranges' growing in it, the species is most likely Citrus sinensis. Oranges are cultivated for their sweet, juicy fruits, enjoyed fresh or used in various culinary and beverage products."
    },
    213: {
        "Crop_Name": 'Honeydew Melons',
        "Likely_Crop_Species": ['Cucumis melo'],
        "Discussion": "When a field has 'honeydew melons' growing in it, the species is most likely Cucumis melo. Honeydew melons are cultivated for their sweet, juicy fruits, enjoyed fresh or used in various culinary dishes."
    },
    214: {
        "Crop_Name": 'Broccoli',
        "Likely_Crop_Species": ['Brassica oleracea var. italica'],
        "Discussion": "When a field has 'broccoli' growing in it, the species is most likely Brassica oleracea var. italica. Broccoli is cultivated for its edible flower heads, used in a variety of culinary dishes."
    },
    215: {
        "Crop_Name": 'Avocados',
        "Likely_Crop_Species": ['Persea americana'],
        "Discussion": "When a field has 'avocados' growing in it, the species is most likely Persea americana. Avocados are cultivated for their creamy, nutrient-rich fruits, enjoyed fresh or used in various culinary dishes."
    },
    216: {
        "Crop_Name": 'Peppers',
        "Likely_Crop_Species": ['Capsicum annuum', 'Capsicum frutescens', 'Capsicum chinense'],
        "Discussion": "When a field has 'peppers' growing in it, the species are most likely Capsicum annuum, Capsicum frutescens, or Capsicum chinense. Peppers are cultivated for their edible fruits, used fresh or dried in various culinary dishes."
    },
    217: {
        "Crop_Name": 'Pomegranates',
        "Likely_Crop_Species": ['Punica granatum'],
        "Discussion": "When a field has 'pomegranates' growing in it, the species is most likely Punica granatum. Pomegranates are cultivated for their edible fruits, known for their juicy, flavorful arils."
    },
    218: {
        "Crop_Name": 'Nectarines',
        "Likely_Crop_Species": ['Prunus persica var. nucipersica'],
        "Discussion": "When a field has 'nectarines' growing in it, the species is most likely Prunus persica var. nucipersica. Nectarines are a smooth-skinned variety of peach, enjoyed fresh or used in various culinary dishes."
    },
    219: {
        "Crop_Name": 'Greens',
        "Likely_Crop_Species": ['Various species'],
        "Discussion": "When a field has 'greens' growing in it, it includes a variety of leafy vegetable species such as spinach (Spinacia oleracea), kale (Brassica oleracea var. sabellica), and collards (Brassica oleracea var. viridis). Greens are cultivated for their edible leaves, used in a variety of culinary dishes."
    },
    220: {
        "Crop_Name": 'Plums',
        "Likely_Crop_Species": ['Prunus domestica'],
        "Discussion": "When a field has 'plums' growing in it, the species is most likely Prunus domestica. Plums are cultivated for their sweet, juicy fruits, enjoyed fresh or used in various culinary dishes and desserts."
    },
    221: {
        "Crop_Name": 'Strawberries',
        "Likely_Crop_Species": ['Fragaria × ananassa'],
        "Discussion": "When a field has 'strawberries' growing in it, the species is most likely Fragaria × ananassa. Strawberries are cultivated for their sweet, juicy fruits, enjoyed fresh or used in various culinary dishes and desserts."
    },
    222: {
        "Crop_Name": 'Squash',
        "Likely_Crop_Species": ['Cucurbita pepo', 'Cucurbita moschata', 'Cucurbita maxima'],
        "Discussion": "When a field has 'squash' growing in it, the species are most likely Cucurbita pepo (summer squash, zucchini), Cucurbita moschata (butternut squash), or Cucurbita maxima (winter squash, pumpkins). Squash is cultivated for its edible fruits, used in a variety of culinary dishes."
    },
    223: {
        "Crop_Name": 'Apricots',
        "Likely_Crop_Species": ['Prunus armeniaca'],
        "Discussion": "When a field has 'apricots' growing in it, the species is most likely Prunus armeniaca. Apricots are cultivated for their sweet, juicy fruits, enjoyed fresh or used in various culinary dishes and desserts."
    },
    224: {
        "Crop_Name": 'Vetch',
        "Likely_Crop_Species": ['Vicia sativa', 'Vicia villosa'],
        "Discussion": "When a field has 'vetch' growing in it, the species are most likely Vicia sativa (common vetch) or Vicia villosa (hairy vetch). Vetch is cultivated as a cover crop, forage, and green manure."
    },
    225: {
        "Crop_Name": 'Dbl Crop WinWht/Corn',
        "Likely_Crop_Species": ['Triticum aestivum', 'Zea mays'],
        "Discussion": "When a field has 'Dbl Crop WinWht/Corn' growing in it, the species are most likely Triticum aestivum (winter wheat) and Zea mays (corn). Double cropping involves growing two different crops in the same field in one year, usually winter wheat followed by corn."
    },
    226: {
        "Crop_Name": 'Dbl Crop Oats/Corn',
        "Likely_Crop_Species": ['Avena sativa', 'Zea mays'],
        "Discussion": "When a field has 'Dbl Crop Oats/Corn' growing in it, the species are most likely Avena sativa (oats) and Zea mays (corn). Double cropping involves growing two different crops in the same field in one year, usually oats followed by corn."
    },
    227: {
        "Crop_Name": 'Lettuce',
        "Likely_Crop_Species": ['Lactuca sativa'],
        "Discussion": "When a field has 'lettuce' growing in it, the species is most likely Lactuca sativa. Lettuce is cultivated for its edible leaves, used fresh in salads and various culinary dishes."
    },
    228: {
        "Crop_Name": 'Dbl Crop Triticale/Corn',
        "Likely_Crop_Species": ['× Triticosecale', 'Zea mays'],
        "Discussion": "When a field has 'Dbl Crop Triticale/Corn' growing in it, the species are most likely × Triticosecale (triticale) and Zea mays (corn). Double cropping involves growing two different crops in the same field in one year, usually triticale followed by corn."
    },
    229: {
        "Crop_Name": 'Pumpkins',
        "Likely_Crop_Species": ['Cucurbita pepo', 'Cucurbita moschata', 'Cucurbita maxima'],
        "Discussion": "When a field has 'pumpkins' growing in it, the species are most likely Cucurbita pepo, Cucurbita moschata, or Cucurbita maxima. Pumpkins are cultivated for their edible fruits, used in a variety of culinary dishes and as decorations."
    },
    230: {
        "Crop_Name": 'Dbl Crop Lettuce/Durum Wht',
        "Likely_Crop_Species": ['Lactuca sativa', 'Triticum durum'],
        "Discussion": "When a field has 'Dbl Crop Lettuce/Durum Wht' growing in it, the species are most likely Lactuca sativa (lettuce) and Triticum durum (durum wheat). Double cropping involves growing two different crops in the same field in one year, usually lettuce followed by durum wheat."
    },
    231: {
        "Crop_Name": 'Dbl Crop Lettuce/Cantaloupe',
        "Likely_Crop_Species": ['Lactuca sativa', 'Cucumis melo'],
        "Discussion": "When a field has 'Dbl Crop Lettuce/Cantaloupe' growing in it, the species are most likely Lactuca sativa (lettuce) and Cucumis melo (cantaloupe). Double cropping involves growing two different crops in the same field in one year, usually lettuce followed by cantaloupe."
    },
    232: {
        "Crop_Name": 'Dbl Crop Lettuce/Cotton',
        "Likely_Crop_Species": ['Lactuca sativa', 'Gossypium hirsutum'],
        "Discussion": "When a field has 'Dbl Crop Lettuce/Cotton' growing in it, the species are most likely Lactuca sativa (lettuce) and Gossypium hirsutum (cotton). Double cropping involves growing two different crops in the same field in one year, usually lettuce followed by cotton."
    },
    233: {
        "Crop_Name": 'Dbl Crop Lettuce/Barley',
        "Likely_Crop_Species": ['Lactuca sativa', 'Hordeum vulgare'],
        "Discussion": "When a field has 'Dbl Crop Lettuce/Barley' growing in it, the species are most likely Lactuca sativa (lettuce) and Hordeum vulgare (barley). Double cropping involves growing two different crops in the same field in one year, usually lettuce followed by barley."
    },
    234: {
        "Crop_Name": 'Dbl Crop Durum Wht/Sorghum',
        "Likely_Crop_Species": ['Triticum durum', 'Sorghum bicolor'],
        "Discussion": "When a field has 'Dbl Crop Durum Wht/Sorghum' growing in it, the species are most likely Triticum durum (durum wheat) and Sorghum bicolor (sorghum). Double cropping involves growing two different crops in the same field in one year, usually durum wheat followed by sorghum."
    },
    235: {
        "Crop_Name": 'Dbl Crop Barley/Sorghum',
        "Likely_Crop_Species": ['Hordeum vulgare', 'Sorghum bicolor'],
            "Discussion": "When a field has 'Dbl Crop Barley/Sorghum' growing in it, the species are most likely Hordeum vulgare (barley) and Sorghum bicolor (sorghum). Double cropping involves growing two different crops in the same field in one year, usually barley followed by sorghum."
    },
    236: {
        "Crop_Name": 'Dbl Crop WinWht/Sorghum',
        "Likely_Crop_Species": ['Triticum aestivum', 'Sorghum bicolor'],
        "Discussion": "When a field has 'Dbl Crop WinWht/Sorghum' growing in it, the species are most likely Triticum aestivum (winter wheat) and Sorghum bicolor (sorghum). Double cropping involves growing two different crops in the same field in one year, usually winter wheat followed by sorghum."
    },
    237: {
        "Crop_Name": 'Dbl Crop Barley/Corn',
        "Likely_Crop_Species": ['Hordeum vulgare', 'Zea mays'],
        "Discussion": "When a field has 'Dbl Crop Barley/Corn' growing in it, the species are most likely Hordeum vulgare (barley) and Zea mays (corn). Double cropping involves growing two different crops in the same field in one year, usually barley followed by corn."
    },
    238: {
        "Crop_Name": 'Dbl Crop WinWht/Cotton',
        "Likely_Crop_Species": ['Triticum aestivum', 'Gossypium hirsutum'],
        "Discussion": "When a field has 'Dbl Crop WinWht/Cotton' growing in it, the species are most likely Triticum aestivum (winter wheat) and Gossypium hirsutum (cotton). Double cropping involves growing two different crops in the same field in one year, usually winter wheat followed by cotton."
    },
    239: {
        "Crop_Name": 'Dbl Crop Soybeans/Cotton',
        "Likely_Crop_Species": ['Glycine max', 'Gossypium hirsutum'],
        "Discussion": "When a field has 'Dbl Crop Soybeans/Cotton' growing in it, the species are most likely Glycine max (soybeans) and Gossypium hirsutum (cotton). Double cropping involves growing two different crops in the same field in one year, usually soybeans followed by cotton."
    },
    240: {
        "Crop_Name": 'Dbl Crop Soybeans/Oats',
        "Likely_Crop_Species": ['Glycine max', 'Avena sativa'],
        "Discussion": "When a field has 'Dbl Crop Soybeans/Oats' growing in it, the species are most likely Glycine max (soybeans) and Avena sativa (oats). Double cropping involves growing two different crops in the same field in one year, usually soybeans followed by oats."
    },
    241: {
        "Crop_Name": 'Dbl Crop Corn/Soybeans',
        "Likely_Crop_Species": ['Zea mays', 'Glycine max'],
        "Discussion": "When a field has 'Dbl Crop Corn/Soybeans' growing in it, the species are most likely Zea mays (corn) and Glycine max (soybeans). Double cropping involves growing two different crops in the same field in one year, usually corn followed by soybeans."
    },
    242: {
        "Crop_Name": 'Blueberries',
        "Likely_Crop_Species": ['Vaccinium corymbosum'],
        "Discussion": "When a field has 'blueberries' growing in it, the species is most likely Vaccinium corymbosum. Blueberries are cultivated for their sweet, nutrient-rich fruits, enjoyed fresh or used in various culinary dishes and desserts."
    },
    243: {
        "Crop_Name": 'Cabbage',
        "Likely_Crop_Species": ['Brassica oleracea var. capitata'],
        "Discussion": "When a field has 'cabbage' growing in it, the species is most likely Brassica oleracea var. capitata. Cabbage is cultivated for its dense, leafy heads, used in a variety of culinary dishes."
    },
    244: {
        "Crop_Name": 'Cauliflower',
        "Likely_Crop_Species": ['Brassica oleracea var. botrytis'],
        "Discussion": "When a field has 'cauliflower' growing in it, the species is most likely Brassica oleracea var. botrytis. Cauliflower is cultivated for its edible flower heads, used in a variety of culinary dishes."
    },
    245: {
        "Crop_Name": 'Celery',
        "Likely_Crop_Species": ['Apium graveolens'],
        "Discussion": "When a field has 'celery' growing in it, the species is most likely Apium graveolens. Celery is cultivated for its edible stalks and leaves, used in a variety of culinary dishes."
    },
    246: {
        "Crop_Name": 'Radishes',
        "Likely_Crop_Species": ['Raphanus sativus'],
        "Discussion": "When a field has 'radishes' growing in it, the species is most likely Raphanus sativus. Radishes are cultivated for their edible roots, used fresh in salads and various culinary dishes."
    },
    247: {
        "Crop_Name": 'Turnips',
        "Likely_Crop_Species": ['Brassica rapa subsp. rapa'],
        "Discussion": "When a field has 'turnips' growing in it, the species is most likely Brassica rapa subsp. rapa. Turnips are cultivated for their edible roots and greens, used in a variety of culinary dishes."
    },
    248: {
        "Crop_Name": 'Eggplants',
        "Likely_Crop_Species": ['Solanum melongena'],
        "Discussion": "When a field has 'eggplants' growing in it, the species is most likely Solanum melongena. Eggplants are cultivated for their edible fruits, used in a variety of culinary dishes."
    },
    249: {
        "Crop_Name": 'Gourds',
        "Likely_Crop_Species": ['Cucurbita pepo', 'Lagenaria siceraria'],
        "Discussion": "When a field has 'gourds' growing in it, the species are most likely Cucurbita pepo or Lagenaria siceraria. Gourds are cultivated for their ornamental fruits, used as decorations or in crafts."
    },
    250: {
        "Crop_Name": 'Cranberries',
        "Likely_Crop_Species": ['Vaccinium macrocarpon'],
        "Discussion": "When a field has 'cranberries' growing in it, the species is most likely Vaccinium macrocarpon. Cranberries are cultivated for their tart, nutrient-rich fruits, used in a variety of culinary dishes and beverages."
    },
    254: {
        "Crop_Name": 'Dbl Crop Barley/Soybeans',
        "Likely_Crop_Species": ['Hordeum vulgare', 'Glycine max'],
        "Discussion": "When a field has 'Dbl Crop Barley/Soybeans' growing in it, the species are most likely Hordeum vulgare (barley) and Glycine max (soybeans). Double cropping involves growing two different crops in the same field in one year, usually barley followed by soybeans."
    }
}

In [10]:
cdl_codes_species_df = pd.DataFrame.from_dict(cdl_codes_species).T
cdl_codes_species_df

,Crop_Name,Likely_Crop_Species,Discussion
1,Corn,[Zea mays],"When a field has 'corn' growing in it, the spe..."
2,Cotton,"[Gossypium hirsutum, Gossypium barbadense, Gos...","When a field has 'cotton' growing in it, the s..."
3,Rice,"[Oryza sativa, Oryza glaberrima]","When a field has 'rice' growing in it, the spe..."
4,Sorghum,[Sorghum bicolor],"When a field has 'sorghum' growing in it, the ..."
5,Soybeans,[Glycine max],"When a field has 'soybeans' growing in it, the..."
...,...,...,...
247,Turnips,[Brassica rapa subsp. rapa],"When a field has 'turnips' growing in it, the ..."
248,Eggplants,[Solanum melongena],"When a field has 'eggplants' growing in it, th..."
249,Gourds,"[Cucurbita pepo, Lagenaria siceraria]","When a field has 'gourds' growing in it, the s..."
250,Cranberries,[Vaccinium macrocarpon],"When a field has 'cranberries' growing in it, ..."


In [11]:
cdl_codes_species_df['CDL_Code'] = csb_crops['Codes']

In [12]:
cdl_codes_species_df = cdl_codes_species_df[['CDL_Code', 'Crop_Name', 'Likely_Crop_Species', 'Discussion']]
cdl_codes_species_df

,CDL_Code,Crop_Name,Likely_Crop_Species,Discussion
1,1,Corn,[Zea mays],"When a field has 'corn' growing in it, the spe..."
2,2,Cotton,"[Gossypium hirsutum, Gossypium barbadense, Gos...","When a field has 'cotton' growing in it, the s..."
3,3,Rice,"[Oryza sativa, Oryza glaberrima]","When a field has 'rice' growing in it, the spe..."
4,4,Sorghum,[Sorghum bicolor],"When a field has 'sorghum' growing in it, the ..."
5,5,Soybeans,[Glycine max],"When a field has 'soybeans' growing in it, the..."
...,...,...,...,...
247,247,Turnips,[Brassica rapa subsp. rapa],"When a field has 'turnips' growing in it, the ..."
248,248,Eggplants,[Solanum melongena],"When a field has 'eggplants' growing in it, th..."
249,249,Gourds,"[Cucurbita pepo, Lagenaria siceraria]","When a field has 'gourds' growing in it, the s..."
250,250,Cranberries,[Vaccinium macrocarpon],"When a field has 'cranberries' growing in it, ..."


In [13]:
cdl_codes_species_df['Likely_Crop_Species'][2]

['Gossypium hirsutum',
 'Gossypium barbadense',
 'Gossypium arboreum',
 'Gossypium herbaceum']

In [14]:
cdl_codes_species_df

,CDL_Code,Crop_Name,Likely_Crop_Species,Discussion
1,1,Corn,[Zea mays],"When a field has 'corn' growing in it, the spe..."
2,2,Cotton,"[Gossypium hirsutum, Gossypium barbadense, Gos...","When a field has 'cotton' growing in it, the s..."
3,3,Rice,"[Oryza sativa, Oryza glaberrima]","When a field has 'rice' growing in it, the spe..."
4,4,Sorghum,[Sorghum bicolor],"When a field has 'sorghum' growing in it, the ..."
5,5,Soybeans,[Glycine max],"When a field has 'soybeans' growing in it, the..."
...,...,...,...,...
247,247,Turnips,[Brassica rapa subsp. rapa],"When a field has 'turnips' growing in it, the ..."
248,248,Eggplants,[Solanum melongena],"When a field has 'eggplants' growing in it, th..."
249,249,Gourds,"[Cucurbita pepo, Lagenaria siceraria]","When a field has 'gourds' growing in it, the s..."
250,250,Cranberries,[Vaccinium macrocarpon],"When a field has 'cranberries' growing in it, ..."


In [15]:
cdl_codes_species_df.to_csv('../data/fields/cdl_codes_species_discussion.csv')
# cdl_codes_species_df.to_pickle('../data/public/cdl_codes_species_discussion.pickle')

In [17]:
crops = pd.read_pickle('../data/crops/EcoCrop_Clean_Imputed_All_with_Common_Name_and_Notes.pickle')
crops

,Crop_Code,Scientific_Name,Genus,Species,Variety,Life_Form,Habit,Life_Span,Physiology,Category,...,Use_Detailed,Use_Part,Climate_Zone_Trewartha,USDA_Hardiness_Zone,USDA_Hardiness_Zone_Min,USDA_Hardiness_Zone_Max,Datasheet_URL,PFAF_URL,Common_Name,Notes
0,289,Abelmoschus esculentus,Abelmoschus,esculentus,,herb,erect,annual,single stem,vegetables,...,vitamins,fruits,"tropical wet & dry (Aw), tropical wet (Ar), st...",5-11,5,11,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Abe...,"abelmoskus, america-neri, bakhua mun, bamia, b...","BRIEF DESCRIPTION A herbaceous, upright, ofte..."
1,290,Abelmoschus manihot,Abelmoschus,manihot,,shrub,erect,"annual, perennial","deciduous, multi stem","vegetables, ornamentals/turf, medicinals & aro...",...,vitamins,leaves,"tropical wet & dry (Aw), tropical wet (Ar)",8-11,8,11,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Abe...,"neka (Simbo), bele (Fiji), pele (Tonga, Tuvalu...",BRIEF DESCRIPTION A shallow rooted shrub reac...
2,291,Abelmoschus moschatus,Abelmoschus,moschatus,,"herb, sub-shrub",prostrate/procumbent/semi-erect,"annual, biennial, perennial","deciduous, multi stem","ornamentals/turf, medicinals & aromatic",...,ornamental/turf,entire plant,tropical wet & dry (Aw),8-11,8,11,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Abe...,"abelmosk, musk mallow, mushkdan, muskdana, kas...",DESCRIPTION: It is a coarse herb or prostrate ...
3,295,Acacia auriculiformis,Acacia,auriculiformis,,tree,erect,perennial,single stem,forest/wood,...,dye/tannin,stems,"tropical wet & dry (Aw), tropical wet (Ar)",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Aca...,"Papuan wattle, auri, ear leaf acacia, tuhkehn ...",BRIEF DESCRIPTION A low to medium-sized fast ...
4,297,Acacia farnesiana,Acacia,farnesiana,,tree,erect,perennial,single stem,"materials, ornamentals/turf, medicinals & arom...",...,ornamental/turf,entire plant,"tropical wet & dry (Aw), steppe or semiarid (B...",9-11,9,11,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Aca...,"Ellington curse, klu, cassie, espino blanco, a...",BRIEF DESCRIPTION A thorny bush or small tree...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2563,400000,Chamaecrista rotundifolia,Chamaecrista,rotundifolia,,herb,prostrate/procumbent/semi-erect,"annual, perennial","deciduous, multi stem","forage/pasture, medicinals & aromatic",...,vitamins,entire plant,"tropical wet & dry (Aw), tropical wet (Ar), su...",9-11,9,11,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Cha...,Black catechu,
2564,400001,Acacia polyacantha,Acacia,polyacantha,,"shrub, tree",erect,perennial,"single stem, multi stem","forest/wood, environmental",...,fuelwood,bark,"desert or arid (Bw), steppe or semiarid (Bs)",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Aca...,Algarobilla,
2565,400002,Prosopis affinis,Prosopis,affinis,,tree,erect,perennial,"deciduous, single stem, C3 photosynthesis",forest/wood,...,vitamins,bark,"tropical wet & dry (Aw), desert or arid (Bw), ...",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Pro...,Hairy vetch,
2566,400003,Vicia dasycarpa,Vicia,dasycarpa,,herb,climber/scrambler/scadent,annual,multi stem,forage/pasture,...,minerals,entire plant,"tropical wet & dry (Aw), tropical wet (Ar), st...",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Vic...,"camelina, false flax, gold-of-pleasure, gergel...",DESCRIPTION: It attains heights of 30 to 90 cm...


In [16]:
# fields = gpd.read_parquet('../data/public/csb_sample_with_temperature_data.parquet')
# fields

In [17]:
# fields.crs

In [18]:
# fields.columns

In [18]:
cdl_codes_species_df

,CDL_Code,Crop_Name,Likely_Crop_Species,Discussion
1,1,Corn,[Zea mays],"When a field has 'corn' growing in it, the spe..."
2,2,Cotton,"[Gossypium hirsutum, Gossypium barbadense, Gos...","When a field has 'cotton' growing in it, the s..."
3,3,Rice,"[Oryza sativa, Oryza glaberrima]","When a field has 'rice' growing in it, the spe..."
4,4,Sorghum,[Sorghum bicolor],"When a field has 'sorghum' growing in it, the ..."
5,5,Soybeans,[Glycine max],"When a field has 'soybeans' growing in it, the..."
...,...,...,...,...
247,247,Turnips,[Brassica rapa subsp. rapa],"When a field has 'turnips' growing in it, the ..."
248,248,Eggplants,[Solanum melongena],"When a field has 'eggplants' growing in it, th..."
249,249,Gourds,"[Cucurbita pepo, Lagenaria siceraria]","When a field has 'gourds' growing in it, the s..."
250,250,Cranberries,[Vaccinium macrocarpon],"When a field has 'cranberries' growing in it, ..."


In [19]:
# Create a dictionary mapping scientific names to EcoCrop IDs
mapping_df = crops['Scientific_Name'].copy()
scientific_name_to_id = {v:k for k, v in mapping_df.to_dict().items()}
# scientific_name_to_id

In [20]:
# Use a list comprehension to map species names to their EcoCrop IDs
cdl_codes_species_df = cdl_codes_species_df.copy()
cdl_codes_species_df['Likely_EcoCrop_IDs'] = [
    [scientific_name_to_id[species] for species in species_list if species in scientific_name_to_id]
    for species_list in cdl_codes_species_df['Likely_Crop_Species']
]
cdl_codes_species_df

,CDL_Code,Crop_Name,Likely_Crop_Species,Discussion,Likely_EcoCrop_IDs
1,1,Corn,[Zea mays],"When a field has 'corn' growing in it, the spe...",[418]
2,2,Cotton,"[Gossypium hirsutum, Gossypium barbadense, Gos...","When a field has 'cotton' growing in it, the s...","[209, 208, 1360, 1361]"
3,3,Rice,"[Oryza sativa, Oryza glaberrima]","When a field has 'rice' growing in it, the spe...","[286, 1605]"
4,4,Sorghum,[Sorghum bicolor],"When a field has 'sorghum' growing in it, the ...",[2281]
5,5,Soybeans,[Glycine max],"When a field has 'soybeans' growing in it, the...",[206]
...,...,...,...,...,...
247,247,Turnips,[Brassica rapa subsp. rapa],"When a field has 'turnips' growing in it, the ...",[]
248,248,Eggplants,[Solanum melongena],"When a field has 'eggplants' growing in it, th...",[360]
249,249,Gourds,"[Cucurbita pepo, Lagenaria siceraria]","When a field has 'gourds' growing in it, the s...","[137, 236]"
250,250,Cranberries,[Vaccinium macrocarpon],"When a field has 'cranberries' growing in it, ...",[2021]


In [21]:
cdl_codes_species_df[cdl_codes_species_df['Likely_EcoCrop_IDs'].apply(len)==0]

,CDL_Code,Crop_Name,Likely_Crop_Species,Discussion,Likely_EcoCrop_IDs
12,12,Sweet Corn,[Zea mays convar. saccharata],"When a field has 'sweet corn' growing in it, t...",[]
37,37,Other Hay/Non Alfalfa,[Various species],When a field has 'other hay/non alfalfa' growi...,[]
44,44,Other Crops,[Various species],"When a field has 'other crops' growing in it, ...",[]
47,47,Misc Vegs & Fruits,[Various species],When a field has 'misc vegs & fruits' growing ...,[]
54,54,Tomatoes,[Solanum lycopersicum],"When a field has 'tomatoes' growing in it, the...",[]
55,55,Caneberries,[Rubus spp.],"When a field has 'caneberries' growing in it, ...",[]
57,57,Herbs,[Various species],"When a field has 'herbs' growing in it, it inc...",[]
70,70,Christmas Trees,[Various species],When a field has 'christmas trees' growing in ...,[]
71,71,Other Tree Crops,[Various species],When a field has 'other tree crops' growing in...,[]
74,74,Pecans,[Carya illinoinensis],"When a field has 'pecans' growing in it, the s...",[]


In [22]:
# Create normalized scientific name in the EcoCrop dataframe
crops['Normalized_Scientific_Name'] = crops['Genus'] + ' ' + crops['Species']
# crops['Normalized_Scientific_Name']

In [23]:
# Function to normalize scientific names by removing subspecies or variety information
def normalize_scientific_name(name):
    return re.sub(r'\s+(subsp\.|var\.).*', '', name)

In [24]:
# Normalize the scientific names in the Likely_Crop_Species column
cdl_codes_species_df['Likely_Crop_Species_Normalized'] = cdl_codes_species_df['Likely_Crop_Species'].apply(lambda species_list: [normalize_scientific_name(species) for species in species_list])
cdl_codes_species_df['Likely_Crop_Species_Normalized']

1                                             [Zea mays]
2      [Gossypium hirsutum, Gossypium barbadense, Gos...
3                       [Oryza sativa, Oryza glaberrima]
4                                      [Sorghum bicolor]
5                                          [Glycine max]
                             ...                        
247                                      [Brassica rapa]
248                                  [Solanum melongena]
249                [Cucurbita pepo, Lagenaria siceraria]
250                              [Vaccinium macrocarpon]
254                       [Hordeum vulgare, Glycine max]
Name: Likely_Crop_Species_Normalized, Length: 106, dtype: object

In [26]:
# cdl_codes_species_df = cdl_codes_species_df.drop(columns=['Normalized_Crop_Species', 'Likely_Normalized_Crop_Species'])

In [25]:
# Create a dictionary mapping scientific names to EcoCrop IDs
mapping_df = crops['Normalized_Scientific_Name'].copy()
scientific_name_to_id = {v:k for k, v in mapping_df.to_dict().items()}
# scientific_name_to_id

In [26]:
# Use a list comprehension to map species names to their EcoCrop IDs
cdl_codes_species_df['Likely_EcoCrop_IDs'] = [
    [scientific_name_to_id[species] for species in species_list if species in scientific_name_to_id]
    for species_list in cdl_codes_species_df['Likely_Crop_Species_Normalized']
]
cdl_codes_species_df

,CDL_Code,Crop_Name,Likely_Crop_Species,Discussion,Likely_EcoCrop_IDs,Likely_Crop_Species_Normalized
1,1,Corn,[Zea mays],"When a field has 'corn' growing in it, the spe...",[2411],[Zea mays]
2,2,Cotton,"[Gossypium hirsutum, Gossypium barbadense, Gos...","When a field has 'cotton' growing in it, the s...","[209, 208, 1360, 1361]","[Gossypium hirsutum, Gossypium barbadense, Gos..."
3,3,Rice,"[Oryza sativa, Oryza glaberrima]","When a field has 'rice' growing in it, the spe...","[1608, 1605]","[Oryza sativa, Oryza glaberrima]"
4,4,Sorghum,[Sorghum bicolor],"When a field has 'sorghum' growing in it, the ...",[2281],[Sorghum bicolor]
5,5,Soybeans,[Glycine max],"When a field has 'soybeans' growing in it, the...",[206],[Glycine max]
...,...,...,...,...,...,...
247,247,Turnips,[Brassica rapa subsp. rapa],"When a field has 'turnips' growing in it, the ...",[869],[Brassica rapa]
248,248,Eggplants,[Solanum melongena],"When a field has 'eggplants' growing in it, th...",[360],[Solanum melongena]
249,249,Gourds,"[Cucurbita pepo, Lagenaria siceraria]","When a field has 'gourds' growing in it, the s...","[137, 236]","[Cucurbita pepo, Lagenaria siceraria]"
250,250,Cranberries,[Vaccinium macrocarpon],"When a field has 'cranberries' growing in it, ...",[2021],[Vaccinium macrocarpon]


In [27]:
cdl_codes_species_df = cdl_codes_species_df[
[
    'CDL_Code', 
    'Crop_Name', 
    'Likely_Crop_Species', 
    'Likely_Crop_Species_Normalized', 
    'Likely_EcoCrop_IDs', 
    'Discussion',
     ]
]
cdl_codes_species_df

,CDL_Code,Crop_Name,Likely_Crop_Species,Likely_Crop_Species_Normalized,Likely_EcoCrop_IDs,Discussion
1,1,Corn,[Zea mays],[Zea mays],[2411],"When a field has 'corn' growing in it, the spe..."
2,2,Cotton,"[Gossypium hirsutum, Gossypium barbadense, Gos...","[Gossypium hirsutum, Gossypium barbadense, Gos...","[209, 208, 1360, 1361]","When a field has 'cotton' growing in it, the s..."
3,3,Rice,"[Oryza sativa, Oryza glaberrima]","[Oryza sativa, Oryza glaberrima]","[1608, 1605]","When a field has 'rice' growing in it, the spe..."
4,4,Sorghum,[Sorghum bicolor],[Sorghum bicolor],[2281],"When a field has 'sorghum' growing in it, the ..."
5,5,Soybeans,[Glycine max],[Glycine max],[206],"When a field has 'soybeans' growing in it, the..."
...,...,...,...,...,...,...
247,247,Turnips,[Brassica rapa subsp. rapa],[Brassica rapa],[869],"When a field has 'turnips' growing in it, the ..."
248,248,Eggplants,[Solanum melongena],[Solanum melongena],[360],"When a field has 'eggplants' growing in it, th..."
249,249,Gourds,"[Cucurbita pepo, Lagenaria siceraria]","[Cucurbita pepo, Lagenaria siceraria]","[137, 236]","When a field has 'gourds' growing in it, the s..."
250,250,Cranberries,[Vaccinium macrocarpon],[Vaccinium macrocarpon],[2021],"When a field has 'cranberries' growing in it, ..."


In [28]:
# list(np.unique(cdl_codes_species_df['Likely_EcoCrop_IDs']))

In [29]:
# Find rows where 'Likely_EcoCrop_IDs' column contains empty lists
empty_lists_df = cdl_codes_species_df[cdl_codes_species_df['Likely_EcoCrop_IDs'].apply(len) == 0]
len(empty_lists_df)

14

In [30]:
empty_lists_df

,CDL_Code,Crop_Name,Likely_Crop_Species,Likely_Crop_Species_Normalized,Likely_EcoCrop_IDs,Discussion
12,12,Sweet Corn,[Zea mays convar. saccharata],[Zea mays convar. saccharata],[],"When a field has 'sweet corn' growing in it, t..."
37,37,Other Hay/Non Alfalfa,[Various species],[Various species],[],When a field has 'other hay/non alfalfa' growi...
44,44,Other Crops,[Various species],[Various species],[],"When a field has 'other crops' growing in it, ..."
47,47,Misc Vegs & Fruits,[Various species],[Various species],[],When a field has 'misc vegs & fruits' growing ...
54,54,Tomatoes,[Solanum lycopersicum],[Solanum lycopersicum],[],"When a field has 'tomatoes' growing in it, the..."
55,55,Caneberries,[Rubus spp.],[Rubus spp.],[],"When a field has 'caneberries' growing in it, ..."
57,57,Herbs,[Various species],[Various species],[],"When a field has 'herbs' growing in it, it inc..."
70,70,Christmas Trees,[Various species],[Various species],[],When a field has 'christmas trees' growing in ...
71,71,Other Tree Crops,[Various species],[Various species],[],When a field has 'other tree crops' growing in...
74,74,Pecans,[Carya illinoinensis],[Carya illinoinensis],[],"When a field has 'pecans' growing in it, the s..."


In [31]:
# List of species for entries with "Various species"
various_species_lists = {
    'Other Small Grains': ['Avena sativa', 'Secale cereale', 'Triticum aestivum'],
    'Other Hay/Non Alfalfa': crops[crops['Genus'] == 'Trifolium']['Scientific_Name'].tolist(),
    # 'Other Crops': ['Various species'],
    # 'Misc Vegs & Fruits': ['Various species'],
    'Herbs': ['Ocimum basilicum', 'Petroselinum crispum', 'Thymus vulgaris'],
    # 'Christmas Trees': ['Abies spp.', 'Picea spp.', 'Pinus spp.'],
    # 'Other Tree Crops': ['Var|}ious species'],
    'Greens': ['Spinacia oleracea', 'Brassica oleracea var. sabellica', 'Brassica oleracea var. viridis']
}


In [34]:
# # Normalize the scientific names in the Likely_Crop_Species column
# cdl_codes_species_df['Likely_Crop_Species_Normalized'] = cdl_codes_species_df['Likely_Crop_Species'].apply(lambda species_list: [normalize_scientific_name(species) for species in species_list])
# cdl_codes_species_df['Likely_Crop_Species_Normalized']

In [32]:
# Function to update Likely_Crop_Species and Likely_Crop_Species_Normalized columns
def update_species(row, various_species_lists):
    crop_name = row['Crop_Name']
    if crop_name in various_species_lists:
        species_list = various_species_lists[crop_name]
        return species_list
    return row['Likely_Crop_Species']

# Apply the function to update the species columns
cdl_codes_species_df['Likely_Crop_Species'] = cdl_codes_species_df.apply(update_species, axis=1, args=(various_species_lists,))
# cdl_codes_species_df['Likely_Crop_Species_Normalized'] = cdl_codes_species_df.apply(update_species, axis=1, args=(various_species_lists,))

# Normalize the scientific names in the Likely_Crop_Species column
cdl_codes_species_df['Likely_Crop_Species_Normalized'] = cdl_codes_species_df['Likely_Crop_Species'].apply(lambda species_list: [normalize_scientific_name(species) for species in species_list])

# # Define function to get EcoCrop IDs for species using the index of the crops DataFrame
# def get_ecocrop_ids(species_list):
#     return [crops[crops['Scientific_Name'] == species].index[0] for species in species_list if not crops[crops['Scientific_Name'] == species].empty]

# # Update Likely_EcoCrop_IDs column
# cdl_codes_species_df['Likely_EcoCrop_IDs'] = cdl_codes_species_df['Likely_Crop_Species'].apply(get_ecocrop_ids)
# Use a list comprehension to map species names to their EcoCrop IDs
cdl_codes_species_df['Likely_EcoCrop_IDs'] = [
    [scientific_name_to_id[species] for species in species_list if species in scientific_name_to_id]
    for species_list in cdl_codes_species_df['Likely_Crop_Species_Normalized']
]
cdl_codes_species_df

,CDL_Code,Crop_Name,Likely_Crop_Species,Likely_Crop_Species_Normalized,Likely_EcoCrop_IDs,Discussion
1,1,Corn,[Zea mays],[Zea mays],[2411],"When a field has 'corn' growing in it, the spe..."
2,2,Cotton,"[Gossypium hirsutum, Gossypium barbadense, Gos...","[Gossypium hirsutum, Gossypium barbadense, Gos...","[209, 208, 1360, 1361]","When a field has 'cotton' growing in it, the s..."
3,3,Rice,"[Oryza sativa, Oryza glaberrima]","[Oryza sativa, Oryza glaberrima]","[1608, 1605]","When a field has 'rice' growing in it, the spe..."
4,4,Sorghum,[Sorghum bicolor],[Sorghum bicolor],[2281],"When a field has 'sorghum' growing in it, the ..."
5,5,Soybeans,[Glycine max],[Glycine max],[206],"When a field has 'soybeans' growing in it, the..."
...,...,...,...,...,...,...
247,247,Turnips,[Brassica rapa subsp. rapa],[Brassica rapa],[869],"When a field has 'turnips' growing in it, the ..."
248,248,Eggplants,[Solanum melongena],[Solanum melongena],[360],"When a field has 'eggplants' growing in it, th..."
249,249,Gourds,"[Cucurbita pepo, Lagenaria siceraria]","[Cucurbita pepo, Lagenaria siceraria]","[137, 236]","When a field has 'gourds' growing in it, the s..."
250,250,Cranberries,[Vaccinium macrocarpon],[Vaccinium macrocarpon],[2021],"When a field has 'cranberries' growing in it, ..."


In [33]:
# Find rows where 'Likely_EcoCrop_IDs' column contains empty lists
empty_lists_df = cdl_codes_species_df[cdl_codes_species_df['Likely_EcoCrop_IDs'].apply(len) == 0]
len(empty_lists_df)

11

In [34]:
empty_lists_df

,CDL_Code,Crop_Name,Likely_Crop_Species,Likely_Crop_Species_Normalized,Likely_EcoCrop_IDs,Discussion
12,12,Sweet Corn,[Zea mays convar. saccharata],[Zea mays convar. saccharata],[],"When a field has 'sweet corn' growing in it, t..."
44,44,Other Crops,[Various species],[Various species],[],"When a field has 'other crops' growing in it, ..."
47,47,Misc Vegs & Fruits,[Various species],[Various species],[],When a field has 'misc vegs & fruits' growing ...
54,54,Tomatoes,[Solanum lycopersicum],[Solanum lycopersicum],[],"When a field has 'tomatoes' growing in it, the..."
55,55,Caneberries,[Rubus spp.],[Rubus spp.],[],"When a field has 'caneberries' growing in it, ..."
70,70,Christmas Trees,[Various species],[Various species],[],When a field has 'christmas trees' growing in ...
71,71,Other Tree Crops,[Various species],[Various species],[],When a field has 'other tree crops' growing in...
74,74,Pecans,[Carya illinoinensis],[Carya illinoinensis],[],"When a field has 'pecans' growing in it, the s..."
75,75,Almonds,[Prunus dulcis],[Prunus dulcis],[],"When a field has 'almonds' growing in it, the ..."
205,205,Triticale,[× Triticosecale],[× Triticosecale],[],"When a field has 'triticale' growing in it, th..."


In [35]:
# empty_lists_df.to_dict()

In [36]:
emtpy_list_manual_fix_d = {
    'CDL_Code': {
        12: 12,
        44: 44,
        47: 47,
        54: 54,
        55: 55,
        70: 70,
        71: 71,
        74: 74,
        75: 75,
        205: 205,
        221: 221
    },
    'Crop_Name': {
        12: 'Sweet Corn',
        44: 'Other Crops',
        47: 'Misc Vegs & Fruits',
        54: 'Tomatoes',
        55: 'Caneberries',
        70: 'Christmas Trees',
        71: 'Other Tree Crops',
        74: 'Pecans',
        
        75: 'Almonds',
        
        205: 'Triticale',
        
        221: 'Strawberries'
    },
    'Likely_Crop_Species': {
        12: ['Zea mays convar. saccharata'],
        44: ['Various species'],
        47: ['Various species'],
        54: ['Solanum lycopersicum', 'Cyphomandra betacea','Lycopersicon esculentum','Physalis philadelphica','Solanum aethiopicum', 'Solanum macrocarpon'],
        
        55: ['Rubus idaeus ssp. idaeus', 'Rubus sp.', 'Rubus fruticosus', 'Rubus occidentalis', 'Rubus occidentalis x idaeus', ],
        
        70: ['Abies alba', 'Abies balsamea', 'Abies concolor', 'Abies pindrow', 'Abies amabilis', 'Picea abies', 'Picea morida', 'Pinus palustris', 'Pinus ponderosa', 'Pinus sylvestris', 'Pinus sylvestris var. mongolica'],
        71: ['Various species'],
        
        74: ['Carya illinoinensis'],
        75: ['Prunus amygdalus'],
        205: ['× Triticosecale'],
        221: ['Fragaria × ananassa' , 'Fragaria vesca', 'Fragaria chiloensis', 'Fragaria virginiana']
    },
    'Likely_Crop_Species_Normalized': {
        12: ['Zea mays'],
        44: ['Various species'],
        47: ['Various species'],
        54: ['Solanum lycopersicum', 'Cyphomandra betacea','Lycopersicon esculentum','Physalis philadelphica','Solanum aethiopicum', 'Solanum macrocarpon'],
        
        55: ['Rubus idaeus', 'Rubus', 'Rubus fruticosus', 'Rubus occidentalis', 'Rubus occidentalis', ],
        
        70: ['Abies alba', 'Abies balsamea', 'Abies concolor', 'Abies pindrow', 'Abies amabilis', 'Picea abies', 'Picea morida', 'Pinus palustris', 'Pinus ponderosa', 'Pinus sylvestris', 'Pinus sylvestris'],
        71: ['Various species'],
        74: ['Carya illinoinensis'],
        75: ['Prunus amygdalus'],
        205: ['× Triticosecale'],
        221: ['Fragaria × ananassa' , 'Fragaria vesca', 'Fragaria chiloensis', 'Fragaria virginiana']
    },
    'Likely_EcoCrop_IDs': {
        12:  [2068],
        44:  [],
        47:  [],
        54:  [145, 247, 1672, 1879, 1885],
        
        55:  [346, 347, 1809, 1810, 1811],
        
        70:  [509, 510, 511, 512, 2369, 1675, 1676, 506, 508, 1708, 1709],
        71:  [],
        74:  [434],
        75:  [469],
        205: [404, 2001, 2002, 2003, 2004, 353, 1851, 2550],
        221: [201, 444, 1338, 1339]
    },
    'Discussion': {
        
        12: "When a field has 'sweet corn' growing in it, the species is most likely Zea mays convar. saccharata. Sweet corn is a variety of maize with a high sugar content, commonly consumed as a vegetable.",
        
        44: "When a field has 'other crops' growing in it, it includes a wide range of species cultivated for various purposes, including vegetables, fruits, and specialty crops.",
        
        47: "When a field has 'misc vegs & fruits' growing in it, it includes a variety of vegetable and fruit species cultivated for food production.",
        
        54: "When a field has 'tomatoes' growing in it, the species is most likely Solanum lycopersicum. Other species may include tree tomatoes/tomarillos (Cyphomandra betacea), tomatillo (Physalis philadelphica), scarlet eggplant/Ethiopian eggplant (Solanum aethiopicum), gboma eggplant (Solanum macrocarpon). Tomatoes are widely cultivated for their edible fruits, used in a variety of culinary dishes.",
        
        55: "When a field has 'caneberries' growing in it, the species are most likely various Rubus species, including blackberries and raspberries. Other species are European raspberry (Rubus idaeus ssp. idaeus) and black raspberry (Rubus occidentalis) . Caneberries are grown for their edible fruits, enjoyed fresh or used in jams and desserts.",
        
        70: "When a field has 'christmas trees' growing in it, it includes a variety of coniferous tree species such as fir (Abies spp.), spruce (Picea spp.), and pine (Pinus spp.). These trees are grown for use as Christmas decorations.",
        
        71: "When a field has 'other tree crops' growing in it, it includes a variety of tree species cultivated for their fruits, nuts, or other products.",
        
        74: "When a field has 'pecans' growing in it, the species is most likely Carya illinoinensis. Pecans are cultivated for their edible nuts, enjoyed fresh or used in various culinary dishes and confections.",
        
        75: "When a field has 'almonds' growing in it, the species is most likely Prunus dulcis. Almonds are grown for their edible seeds, enjoyed fresh or used in various culinary dishes and confections.",
        
        205: "When a field has 'triticale' growing in it, the species is most likely × Triticosecale. Triticale is a hybrid of wheat (Triticum) and rye (Secale), grown for its grain and forage.",
        
        221: "When a field has 'strawberries' growing in it, the species is most likely Fragaria × ananassa. Other species may include wild strawberries (Fragaria vesca), Chilean strawberries (Fragaria chiloensis), and Virginia strawberries (Fragaria virginiana). Strawberries are cultivated for their sweet, juicy fruits, enjoyed fresh or used in various culinary dishes and desserts."
    }
}

In [37]:
emtpy_list_manual_fix_df = pd.DataFrame.from_dict(emtpy_list_manual_fix_d)
emtpy_list_manual_fix_df

,CDL_Code,Crop_Name,Likely_Crop_Species,Likely_Crop_Species_Normalized,Likely_EcoCrop_IDs,Discussion
12,12,Sweet Corn,[Zea mays convar. saccharata],[Zea mays],[2068],"When a field has 'sweet corn' growing in it, t..."
44,44,Other Crops,[Various species],[Various species],[],"When a field has 'other crops' growing in it, ..."
47,47,Misc Vegs & Fruits,[Various species],[Various species],[],When a field has 'misc vegs & fruits' growing ...
54,54,Tomatoes,"[Solanum lycopersicum, Cyphomandra betacea, Ly...","[Solanum lycopersicum, Cyphomandra betacea, Ly...","[145, 247, 1672, 1879, 1885]","When a field has 'tomatoes' growing in it, the..."
55,55,Caneberries,"[Rubus idaeus ssp. idaeus, Rubus sp., Rubus fr...","[Rubus idaeus, Rubus, Rubus fruticosus, Rubus ...","[346, 347, 1809, 1810, 1811]","When a field has 'caneberries' growing in it, ..."
70,70,Christmas Trees,"[Abies alba, Abies balsamea, Abies concolor, A...","[Abies alba, Abies balsamea, Abies concolor, A...","[509, 510, 511, 512, 2369, 1675, 1676, 506, 50...",When a field has 'christmas trees' growing in ...
71,71,Other Tree Crops,[Various species],[Various species],[],When a field has 'other tree crops' growing in...
74,74,Pecans,[Carya illinoinensis],[Carya illinoinensis],[434],"When a field has 'pecans' growing in it, the s..."
75,75,Almonds,[Prunus amygdalus],[Prunus amygdalus],[469],"When a field has 'almonds' growing in it, the ..."
205,205,Triticale,[× Triticosecale],[× Triticosecale],"[404, 2001, 2002, 2003, 2004, 353, 1851, 2550]","When a field has 'triticale' growing in it, th..."


In [38]:
cdl_codes_species_fix_df = cdl_codes_species_df.copy()
cdl_codes_species_fix_df.update(emtpy_list_manual_fix_df)
cdl_codes_species_fix_df

,CDL_Code,Crop_Name,Likely_Crop_Species,Likely_Crop_Species_Normalized,Likely_EcoCrop_IDs,Discussion
1,1,Corn,[Zea mays],[Zea mays],[2411],"When a field has 'corn' growing in it, the spe..."
2,2,Cotton,"[Gossypium hirsutum, Gossypium barbadense, Gos...","[Gossypium hirsutum, Gossypium barbadense, Gos...","[209, 208, 1360, 1361]","When a field has 'cotton' growing in it, the s..."
3,3,Rice,"[Oryza sativa, Oryza glaberrima]","[Oryza sativa, Oryza glaberrima]","[1608, 1605]","When a field has 'rice' growing in it, the spe..."
4,4,Sorghum,[Sorghum bicolor],[Sorghum bicolor],[2281],"When a field has 'sorghum' growing in it, the ..."
5,5,Soybeans,[Glycine max],[Glycine max],[206],"When a field has 'soybeans' growing in it, the..."
...,...,...,...,...,...,...
247,247,Turnips,[Brassica rapa subsp. rapa],[Brassica rapa],[869],"When a field has 'turnips' growing in it, the ..."
248,248,Eggplants,[Solanum melongena],[Solanum melongena],[360],"When a field has 'eggplants' growing in it, th..."
249,249,Gourds,"[Cucurbita pepo, Lagenaria siceraria]","[Cucurbita pepo, Lagenaria siceraria]","[137, 236]","When a field has 'gourds' growing in it, the s..."
250,250,Cranberries,[Vaccinium macrocarpon],[Vaccinium macrocarpon],[2021],"When a field has 'cranberries' growing in it, ..."


In [39]:
# Find rows where 'Likely_EcoCrop_IDs' column contains empty lists
empty_lists_fixed1_df = cdl_codes_species_fix_df[cdl_codes_species_fix_df['Likely_EcoCrop_IDs'].apply(len) == 0]
len(empty_lists_fixed1_df)
empty_lists_fixed1_df

,CDL_Code,Crop_Name,Likely_Crop_Species,Likely_Crop_Species_Normalized,Likely_EcoCrop_IDs,Discussion
44,44,Other Crops,[Various species],[Various species],[],"When a field has 'other crops' growing in it, ..."
47,47,Misc Vegs & Fruits,[Various species],[Various species],[],When a field has 'misc vegs & fruits' growing ...
71,71,Other Tree Crops,[Various species],[Various species],[],When a field has 'other tree crops' growing in...


## save cdl to ecocrop code translator

In [40]:
cdl_codes_species_fix_df

,CDL_Code,Crop_Name,Likely_Crop_Species,Likely_Crop_Species_Normalized,Likely_EcoCrop_IDs,Discussion
1,1,Corn,[Zea mays],[Zea mays],[2411],"When a field has 'corn' growing in it, the spe..."
2,2,Cotton,"[Gossypium hirsutum, Gossypium barbadense, Gos...","[Gossypium hirsutum, Gossypium barbadense, Gos...","[209, 208, 1360, 1361]","When a field has 'cotton' growing in it, the s..."
3,3,Rice,"[Oryza sativa, Oryza glaberrima]","[Oryza sativa, Oryza glaberrima]","[1608, 1605]","When a field has 'rice' growing in it, the spe..."
4,4,Sorghum,[Sorghum bicolor],[Sorghum bicolor],[2281],"When a field has 'sorghum' growing in it, the ..."
5,5,Soybeans,[Glycine max],[Glycine max],[206],"When a field has 'soybeans' growing in it, the..."
...,...,...,...,...,...,...
247,247,Turnips,[Brassica rapa subsp. rapa],[Brassica rapa],[869],"When a field has 'turnips' growing in it, the ..."
248,248,Eggplants,[Solanum melongena],[Solanum melongena],[360],"When a field has 'eggplants' growing in it, th..."
249,249,Gourds,"[Cucurbita pepo, Lagenaria siceraria]","[Cucurbita pepo, Lagenaria siceraria]","[137, 236]","When a field has 'gourds' growing in it, the s..."
250,250,Cranberries,[Vaccinium macrocarpon],[Vaccinium macrocarpon],[2021],"When a field has 'cranberries' growing in it, ..."


In [42]:
cdl_codes_species_fix_df.to_pickle('../data/crops/cdl_to_ecocrop_code_translator.pickle')
cdl_codes_species_fix_df.to_csv('../data/crops/cdl_to_ecocrop_code_translator.csv')
# cdl_codes_species_fix_df.to_pickle('../data/public/cdl_to_ecocrop_code_translator.pickle')

## query crops dataframe for manual fix

In [43]:
crops[crops['Scientific_Name'].str.contains('saccharata')]

,Crop_Code,Scientific_Name,Genus,Species,Variety,Life_Form,Habit,Life_Span,Physiology,Category,...,Use_Part,Climate_Zone_Trewartha,USDA_Hardiness_Zone,USDA_Hardiness_Zone_Min,USDA_Hardiness_Zone_Max,Datasheet_URL,PFAF_URL,Common_Name,Notes,Normalized_Scientific_Name
2068,10981,Zea mays ssp. saccharata,Zea,mays,,grass,erect,annual,single stem,"forage/pasture, vegetables",...,fruits,"tropical wet & dry (Aw), steppe or semiarid (B...",2-11,2,11,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Zea...,"sweet corn, sweet maize, milho doce",BRIEF DESCRIPTION\nA grass and grain crop.\nUS...,Zea mays


In [44]:
crops[crops['Common_Name'].str.contains('tomato')]

,Crop_Code,Scientific_Name,Genus,Species,Variety,Life_Form,Habit,Life_Span,Physiology,Category,...,Use_Part,Climate_Zone_Trewartha,USDA_Hardiness_Zone,USDA_Hardiness_Zone_Min,USDA_Hardiness_Zone_Max,Datasheet_URL,PFAF_URL,Common_Name,Notes,Normalized_Scientific_Name
145,859,Cyphomandra betacea,Cyphomandra,betacea,,"herb, shrub, tree",erect,perennial,"single stem, multi stem","fruits & nuts, materials, medicinals & aromatic",...,fruits,"tropical wet & dry (Aw), tropical wet (Ar), su...",8-11,8,11,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Cyp...,"tree tomato, tamarillo, yellow tree tomato, to...",BRIEF DESCRIPTION A fast-growing herbaceous s...,Cyphomandra betacea
247,1379,Lycopersicon esculentum,Lycopersicon,esculentum,,herb,prostrate/procumbent/semi-erect,annual,single stem,"forage/pasture, vegetables, medicinals & aromatic",...,fruits,"tropical wet & dry (Aw), tropical wet (Ar), st...",8-11,8,11,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Lyc...,"tomato, tomate, pomodoro, tomata, tomatar, tem...","BRIEF DESCRIPTION Variable erect, or spreadin...",Lycopersicon esculentum
1672,8556,Physalis philadelphica,Physalis,philadelphica,,herb,erect,annual,"single stem, multi stem","vegetables, ornamentals/turf",...,fruits,"tropical wet & dry (Aw), tropical wet (Ar)",6-9,6,9,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Phy...,"tomatillo, husk-tomato, jamberry, ground cherr...",BRIEF DESCRIPTION A herb reaching 15-60 cm in...,Physalis philadelphica
1879,9825,Solanum aethiopicum,Solanum,aethiopicum,,"herb, sub-shrub",erect,"annual, perennial","deciduous, multi stem","vegetables, ornamentals/turf",...,fruits,"tropical wet & dry (Aw), tropical wet (Ar), su...",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Sol...,"scarlet eggplant, Ethiopian eggplant, African ...",Temperature: It is frost tender. Reported temp...,Solanum aethiopicum
1885,9878,Solanum macrocarpon,Solanum,macrocarpon,,"herb, vine",erect,perennial,"single stem, multi stem","vegetables, ornamentals/turf",...,fruits,"tropical wet & dry (Aw), tropical wet (Ar), su...",8-11,8,11,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Sol...,"gboma eggplant, African eggplant, anthora, gar...","BRIEF DESCRIPTION A glabrous, erect or climbi...",Solanum macrocarpon


In [45]:
crops[crops['Scientific_Name'].str.contains('Abies')]

,Crop_Code,Scientific_Name,Genus,Species,Variety,Life_Form,Habit,Life_Span,Physiology,Category,...,Use_Part,Climate_Zone_Trewartha,USDA_Hardiness_Zone,USDA_Hardiness_Zone_Min,USDA_Hardiness_Zone_Max,Datasheet_URL,PFAF_URL,Common_Name,Notes,Normalized_Scientific_Name
509,2550,Abies alba,Abies,alba,,tree,erect,perennial,"evergreen, single stem","medicinals & aromatic, forest/wood",...,bark,"temperate oceanic (Do), temperate continental ...",5-8,5,8,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Abi...,"silver fir, abete, abete bianco, Tanne, Weisst...",DESCRIPTION: Tree: Up to 45-55 m tall and 200-...,Abies alba
510,2551,Abies balsamea,Abies,balsamea,,tree,erect,perennial,"evergreen, single stem","materials, forest/wood",...,bark,"temperate oceanic (Do), temperate continental ...",3-6,3,6,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Abi...,"Balsam Fir, Gilead Fir, Fir Balsam, Fir Pine, ...","DESCRIPTION: It is needle leaf, evergreen tree...",Abies balsamea
511,2553,Abies concolor,Abies,concolor,,tree,erect,perennial,"evergreen, single stem","ornamentals/turf, forest/wood, environmental",...,bark,"steppe or semiarid (Bs), subtropical dry summe...",3-7,3,7,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Abi...,"white fir, California white fir, Rocky Mountai...",DESCRIPTION: It is a relatively slow growing l...,Abies concolor
512,2558,Abies pindrow,Abies,pindrow,,tree,erect,perennial,"evergreen, single stem",forest/wood,...,bark,temperate oceanic (Do),5-7,5,7,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Abi...,,"SOURCES\nTroup R 1921 pp 1133-1142 [TEMP, DRA,...",Abies pindrow
2369,196860,Abies amabilis,Abies,amabilis,,tree,erect,perennial,"evergreen, single stem","ornamentals/turf, forest/wood",...,bark,"temperate oceanic (Do), boreal (E)",4-8,4,8,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Abi...,,"DESCRIPTION: Leguminous, erect herb or sub-shr...",Abies amabilis


In [46]:
crops[crops['Scientific_Name'].str.contains('Picea')]

,Crop_Code,Scientific_Name,Genus,Species,Variety,Life_Form,Habit,Life_Span,Physiology,Category,...,Use_Part,Climate_Zone_Trewartha,USDA_Hardiness_Zone,USDA_Hardiness_Zone_Min,USDA_Hardiness_Zone_Max,Datasheet_URL,PFAF_URL,Common_Name,Notes,Normalized_Scientific_Name
1675,8571,Picea abies,Picea,abies,,tree,erect,perennial,"evergreen, single stem","materials, medicinals & aromatic, forest/wood,...",...,bark,"subtropical dry summer (Cs), temperate oceanic...",2-7,2,7,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Pic...,"Norway spruce, white deal, Jura turpentine, Bu...",DESCRIPTION: Norway spruce is an evergreen tre...,Picea abies
1676,8576,Picea morida,Picea,morida,,tree,erect,perennial,"evergreen, single stem","forest/wood, environmental",...,bark,"temperate oceanic (Do), temperate continental ...",2-7,2,7,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Pic...,,"SOURCES\nTroup R 1921 pp 1143-1154 [DRA, FER, ...",Picea morida


In [47]:
crops[crops['Scientific_Name'].str.contains('Pinus')][['Scientific_Name', 'Common_Name']].to_dict()

{'Scientific_Name': {314: 'Pinus caribaea',
  315: 'Pinus merkusii',
  316: 'Pinus michoacana',
  317: 'Pinus oocarpa',
  318: 'Pinus patula',
  319: 'Pinus pseudostrobus',
  320: 'Pinus radiata',
  321: 'Pinus taeda',
  506: 'Pinus palustris',
  507: 'Pinus pinaster',
  508: 'Pinus ponderosa',
  1682: 'Pinus ayacahuite',
  1683: 'Pinus ayachuite var. Br.',
  1684: 'Pinus brutia',
  1685: 'Pinus canariensis',
  1686: 'Pinus caribaea ssp. bahamensis',
  1687: 'Pinus caribaea ssp. caribaea',
  1688: 'Pinus caribaea ssp. hondurensis',
  1689: 'Pinus chiapensis',
  1690: 'Pinus douglasiana',
  1691: 'Pinus durangensis',
  1692: 'Pinus excelsa',
  1693: 'Pinus gerardiana',
  1694: 'Pinus greggii',
  1695: 'Pinus halepensis',
  1696: 'Pinus hartwegii',
  1697: 'Pinus kesiya',
  1698: 'Pinus michoacana var. cornuta',
  1699: 'Pinus montezumae',
  1700: 'Pinus mugo',
  1701: 'Pinus occidentalis',
  1702: 'Pinus oocarpa var. ochoterenai',
  1703: 'Pinus pinea',
  1704: 'Pinus pseudostrobus var.

In [48]:
crops[crops['Scientific_Name'].str.contains('Rubus')]

,Crop_Code,Scientific_Name,Genus,Species,Variety,Life_Form,Habit,Life_Span,Physiology,Category,...,Use_Part,Climate_Zone_Trewartha,USDA_Hardiness_Zone,USDA_Hardiness_Zone_Min,USDA_Hardiness_Zone_Max,Datasheet_URL,PFAF_URL,Common_Name,Notes,Normalized_Scientific_Name
346,1877,Rubus idaeus ssp. idaeus,Rubus,idaeus,,sub-shrub,erect,perennial,"deciduous, multi stem","fruits & nuts, medicinals & aromatic",...,fruits,"steppe or semiarid (Bs), temperate oceanic (Do...",4-8,4,8,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Rub...,"European raspberry, red garden raspberry, red ...",SOURCES\nDube P 1982 pp 18\nDuke J 1975 pp 26 ...,Rubus idaeus
347,1881,Rubus sp.,Rubus,sp.,,shrub,erect,perennial,"deciduous, multi stem","fruits & nuts, medicinals & aromatic",...,fruits,"steppe or semiarid (Bs), temperate oceanic (Do...",3-7,3,7,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Rub...,dewberry,SOURCES\nRoecklein J 1987 pp 212 [USE]\nDuke J...,Rubus sp.
1809,9359,Rubus fruticosus,Rubus,fruticosus,,sub-shrub,climber/scrambler/scadent,perennial,"deciduous, multi stem",medicinals & aromatic,...,fruits,"subtropical dry summer (Cs), temperate oceanic...",5-9,5,9,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Rub...,"blackberry, scaldberry, bramble, ronce commune...",,Rubus fruticosus
1810,9373,Rubus occidentalis,Rubus,occidentalis,,shrub,erect,perennial,"deciduous, multi stem",fruits & nuts,...,fruits,"steppe or semiarid (Bs), temperate oceanic (Do...",3-7,3,7,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Rub...,"black raspberry, blackcap raspberry, framboise...",DESCRIPTION: Deciduous shrub forming arching c...,Rubus occidentalis
1811,9374,Rubus occidentalis x idaeus,Rubus,occidentalis,,shrub,erect,perennial,"deciduous, multi stem",fruits & nuts,...,fruits,"steppe or semiarid (Bs), temperate oceanic (Do...",3-7,3,7,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Rub...,,SOURCES\nSims D (pers. comm.)\nRoecklein J 198...,Rubus occidentalis


In [49]:
crops[crops['Scientific_Name'].str.contains('Carya')] 

,Crop_Code,Scientific_Name,Genus,Species,Variety,Life_Form,Habit,Life_Span,Physiology,Category,...,Use_Part,Climate_Zone_Trewartha,USDA_Hardiness_Zone,USDA_Hardiness_Zone_Min,USDA_Hardiness_Zone_Max,Datasheet_URL,PFAF_URL,Common_Name,Notes,Normalized_Scientific_Name
434,2231,Carya illinoensis,Carya,illinoensis,,tree,erect,perennial,"deciduous, single stem","fruits & nuts, forest/wood",...,seeds,"steppe or semiarid (Bs), subtropical humid (Cf...",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Car...,Pecan,"BRIEF DESCRIPTION A tall, handsome, deciduous...",Carya illinoensis


In [50]:
crops[crops['Common_Name'].str.contains('almond')] 

,Crop_Code,Scientific_Name,Genus,Species,Variety,Life_Form,Habit,Life_Span,Physiology,Category,...,Use_Part,Climate_Zone_Trewartha,USDA_Hardiness_Zone,USDA_Hardiness_Zone_Min,USDA_Hardiness_Zone_Max,Datasheet_URL,PFAF_URL,Common_Name,Notes,Normalized_Scientific_Name
81,606,Canarium indicum,Canarium,indicum,,tree,erect,perennial,single stem,"fruits & nuts, ornamentals/turf, forest/wood",...,seeds,"tropical wet & dry (Aw), tropical wet (Ar)",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Can...,"ngali nut, canarium almond, canarium nut, nut ...",BRIEF DESCRIPTION A tall tree reaching up to ...,Canarium indicum
387,2062,Terminalia catappa,Terminalia,catappa,,tree,erect,perennial,"evergreen, deciduous, single stem","fruits & nuts, materials, ornamentals/turf, fo...",...,seeds,"tropical wet & dry (Aw), tropical wet (Ar)",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Ter...,"Tropical almond, Indian almond, sea almond, ba...",BRIEF DESCRIPTION A small to medium sized eve...,Terminalia catappa
466,2386,Pistacia vera,Pistacia,vera,,tree,erect,perennial,"deciduous, single stem","fruits & nuts, materials, medicinals & aromatic",...,seeds,"steppe or semiarid (Bs), subtropical dry summe...",7-10,7,10,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Pis...,"Pistachio nut, green almond, pistache, pistach...",BRIEF DESCRIPTION A small deciduous tree reac...,Pistacia vera
469,2397,Prunus amygdalus,Prunus,amygdalus,,tree,erect,perennial,"deciduous, single stem","fruits & nuts, medicinals & aromatic",...,bark,"desert or arid (Bw), steppe or semiarid (Bs), ...",3-7,3,7,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Pru...,"almond, sweet almond, amandier, amendoeira, al...","BRIEF DESCRIPTION A small, deciduous, spreadi...",Prunus amygdalus
2088,11641,Prunus africana,Prunus,africana,,tree,erect,perennial,"evergreen, single stem","ornamentals/turf, medicinals & aromatic, fores...",...,entire plant,tropical wet & dry (Aw),10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Pru...,"red stinkwood, African stinkwood, African cher...",DESCRIPTION: It is an evergreen tree reaching ...,Prunus africana
2434,355154,Terminalia brownii,Terminalia,brownii,,tree,erect,perennial,"deciduous, single stem","forage/pasture, materials, ornamentals/turf, m...",...,leaves,"tropical wet & dry (Aw), steppe or semiarid (Bs)",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Ter...,Madagascar almond,DESCRIPTION: It is a deciduous or evergreen tr...,Terminalia brownii


In [51]:
crops[crops['Scientific_Name'].str.contains('dulcis')] 

,Crop_Code,Scientific_Name,Genus,Species,Variety,Life_Form,Habit,Life_Span,Physiology,Category,...,Use_Part,Climate_Zone_Trewartha,USDA_Hardiness_Zone,USDA_Hardiness_Zone_Min,USDA_Hardiness_Zone_Max,Datasheet_URL,PFAF_URL,Common_Name,Notes,Normalized_Scientific_Name
445,2302,Garcinia dulcis,Garcinia,dulcis,,tree,erect,perennial,"evergreen, single stem","fruits & nuts, materials, medicinals & aromatic",...,fruits,"tropical wet & dry (Aw), tropical wet (Ar)",11-12,11,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Gar...,"gourka, egg tree",BRIEF DESCRIPTION A small evergreen tree reac...,Garcinia dulcis
490,2494,Eleocharis dulcis,Eleocharis,dulcis,,herb,erect,annual,multi stem,materials,...,entire plant,"tropical wet & dry (Aw), tropical wet (Ar), su...",9-12,9,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Ele...,Edible spike rush,"BRIEF DESCRIPTION\nA variable, erect \nUSES\nG...",Eleocharis dulcis


In [52]:
crops[crops['Scientific_Name'].str.contains('Triticum')]

,Crop_Code,Scientific_Name,Genus,Species,Variety,Life_Form,Habit,Life_Span,Physiology,Category,...,Use_Part,Climate_Zone_Trewartha,USDA_Hardiness_Zone,USDA_Hardiness_Zone_Min,USDA_Hardiness_Zone_Max,Datasheet_URL,PFAF_URL,Common_Name,Notes,Normalized_Scientific_Name
404,2114,Triticum aestivum,Triticum,aestivum,,grass,erect,annual,"single stem, C3 photosynthesis","cereals & pseudocereals, forage/pasture, medic...",...,seeds,"tropical wet & dry (Aw), tropical wet (Ar), de...",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Tri...,"wheat, bread wheat, blé, alkamh, sinde, xiao m...",DESCRIPTION: Wheat is an hexaploid annual gras...,Triticum aestivum
2001,10579,Triticum compactum,Triticum,compactum,,grass,erect,annual,"single stem, C3 photosynthesis","cereals & pseudocereals, forage/pasture",...,seeds,"tropical wet & dry (Aw), tropical wet (Ar), st...",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Tri...,club wheat,BRIEF DESCRIPTION A grass and cereal crop rea...,Triticum compactum
2002,10581,Triticum dicoccon,Triticum,dicoccon,,grass,erect,annual,"single stem, C3 photosynthesis","cereals & pseudocereals, forage/pasture",...,seeds,"tropical wet & dry (Aw), tropical wet (Ar), st...",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Tri...,,BRIEF DESCRIPTION A grass and cereal crop. US...,Triticum dicoccon
2003,10582,Triticum durum,Triticum,durum,,grass,erect,annual,"single stem, C3 photosynthesis",cereals & pseudocereals,...,seeds,"tropical wet & dry (Aw), tropical wet (Ar), st...",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Tri...,"durum wheat, Macaroni wheat, blé dur, trigo du...",BRIEF DESCRIPTION A grass and cereal crop. US...,Triticum durum
2004,10591,Triticum spelta,Triticum,spelta,,grass,erect,annual,"single stem, C3 photosynthesis","cereals & pseudocereals, forage/pasture",...,seeds,"steppe or semiarid (Bs), temperate oceanic (Do...",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Tri...,"spelt wheat, wheat - spelt, farro, dinkel, Ger...",BRIEF DESCRIPTION\nA grass reaching a height o...,Triticum spelta


In [53]:
crops[crops['Scientific_Name'].str.contains('Secale')]

,Crop_Code,Scientific_Name,Genus,Species,Variety,Life_Form,Habit,Life_Span,Physiology,Category,...,Use_Part,Climate_Zone_Trewartha,USDA_Hardiness_Zone,USDA_Hardiness_Zone_Min,USDA_Hardiness_Zone_Max,Datasheet_URL,PFAF_URL,Common_Name,Notes,Normalized_Scientific_Name
353,1929,Secale cereale,Secale,cereale,,grass,erect,annual,multi stem,"cereals & pseudocereals, forage/pasture, cover...",...,seeds,"steppe or semiarid (Bs), temperate oceanic (Do...",3-7,3,7,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Sec...,"rye, seigle, centeno, roggen, saat-roggen, seg...",BRIEF DESCRIPTION A tufted grass and cereal c...,Secale cereale
1851,9664,Secale montanum,Secale,montanum,,grass,erect,perennial,multi stem,forage/pasture,...,entire plant,"desert or arid (Bw), steppe or semiarid (Bs), ...",3-7,3,7,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Sec...,,BRIEF DESCRIPTION A variable bunchgrass with ...,Secale montanum
2550,376657,Secale africanum,Secale,africanum,,grass,erect,annual,multi stem,"cereals & pseudocereals, forage/pasture, cover...",...,entire plant,"desert or arid (Bw), steppe or semiarid (Bs), ...",3-7,3,7,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Sec...,Purple pigeon grass,,Secale africanum


In [54]:
crops[crops['Genus']=='Trifolium']

,Crop_Code,Scientific_Name,Genus,Species,Variety,Life_Form,Habit,Life_Span,Physiology,Category,...,Use_Part,Climate_Zone_Trewartha,USDA_Hardiness_Zone,USDA_Hardiness_Zone_Min,USDA_Hardiness_Zone_Max,Datasheet_URL,PFAF_URL,Common_Name,Notes,Normalized_Scientific_Name
400,2104,Trifolium pratense,Trifolium,pratense,,herb,erect,"annual, biennial, perennial",multi stem,"forage/pasture, medicinals & aromatic",...,entire plant,"steppe or semiarid (Bs), subtropical humid (Cf...",5-9,5,9,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Tri...,"red clover, broad red clover, broad-leaved clo...",BRIEF DESCRIPTION A herbaceous legume made up...,Trifolium pratense
401,2105,Trifolium repens,Trifolium,repens,,herb,prostrate/procumbent/semi-erect,perennial,multi stem,forage/pasture,...,entire plant,"steppe or semiarid (Bs), subtropical humid (Cf...",4-8,4,8,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Tri...,"white clover, Dutch clover, sheep's gowan, trê...","BRIEF DESCRIPTION A low glabrous, shallow-roo...",Trifolium repens
402,2107,Trifolium semipilosum,Trifolium,semipilosum,,herb,prostrate/procumbent/semi-erect,perennial,multi stem,"forage/pasture, cover crop",...,entire plant,"tropical wet & dry (Aw), steppe or semiarid (B...",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Tri...,"Kenya white clover, Kenya clover, trefle afric...","DESCRIPTION: Herbaceous legume with taproot, i...",Trifolium semipilosum
403,2108,Trifolium usambarense,Trifolium,usambarense,,herb,"erect, prostrate/procumbent/semi-erect","annual, perennial",multi stem,"forage/pasture, materials, environmental",...,entire plant,tropical wet & dry (Aw),10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Tri...,,DESCRIPTION: Herbaceous erect or ascending leg...,Trifolium usambarense
1973,10521,Trifolium agrarium,Trifolium,agrarium,,herb,erect,perennial,multi stem,forage/pasture,...,entire plant,"steppe or semiarid (Bs), subtropical humid (Cf...",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Tri...,,"SOURCES\nHeath M 1985 pp 144 [DRA, FER, DEP, T...",Trifolium agrarium
1974,10522,Trifolium alexandrinum,Trifolium,alexandrinum,,herb,prostrate/procumbent/semi-erect,annual,multi stem,forage/pasture,...,entire plant,"tropical wet & dry (Aw), tropical wet (Ar), st...",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Tri...,"Egyptian clover, berseem clover, trefle d'Alex...",DESCRIPTION: A sparingly hairy annual legume 3...,Trifolium alexandrinum
1975,10524,Trifolium ambiguum,Trifolium,ambiguum,,herb,prostrate/procumbent/semi-erect,perennial,multi stem,"forage/pasture, materials",...,flowers,"steppe or semiarid (Bs), subtropical humid (Cf...",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Tri...,"Caucasian clover, Kura clover, honey clover, P...","BRIEF DESCRIPTION A procumbent to ascending, ...",Trifolium ambiguum
1976,10525,Trifolium arvense,Trifolium,arvense,,herb,prostrate/procumbent/semi-erect,"annual, biennial",multi stem,"forage/pasture, medicinals & aromatic",...,entire plant,"temperate oceanic (Do), temperate continental ...",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Tri...,"hare's-foot clover, rabbit-foot clover, old-fi...","SOURCES\nHeath M 1985 pp 144 [FER, DRA, TEXT]\...",Trifolium arvense
1977,10527,Trifolium campestre,Trifolium,campestre,,herb,prostrate/procumbent/semi-erect,annual,multi stem,forage/pasture,...,entire plant,"subtropical humid (Cf), subtropical dry summer...",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Tri...,"hop clover, hop trefoil, large hop clover, tre...",BRI

In [55]:
crops[crops['Genus']=='Carya']

,Crop_Code,Scientific_Name,Genus,Species,Variety,Life_Form,Habit,Life_Span,Physiology,Category,...,Use_Part,Climate_Zone_Trewartha,USDA_Hardiness_Zone,USDA_Hardiness_Zone_Min,USDA_Hardiness_Zone_Max,Datasheet_URL,PFAF_URL,Common_Name,Notes,Normalized_Scientific_Name
434,2231,Carya illinoensis,Carya,illinoensis,,tree,erect,perennial,"deciduous, single stem","fruits & nuts, forest/wood",...,seeds,"steppe or semiarid (Bs), subtropical humid (Cf...",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Car...,Pecan,"BRIEF DESCRIPTION A tall, handsome, deciduous...",Carya illinoensis


In [56]:
crops[crops['Normalized_Scientific_Name']=='Carya illinoensis']

,Crop_Code,Scientific_Name,Genus,Species,Variety,Life_Form,Habit,Life_Span,Physiology,Category,...,Use_Part,Climate_Zone_Trewartha,USDA_Hardiness_Zone,USDA_Hardiness_Zone_Min,USDA_Hardiness_Zone_Max,Datasheet_URL,PFAF_URL,Common_Name,Notes,Normalized_Scientific_Name
434,2231,Carya illinoensis,Carya,illinoensis,,tree,erect,perennial,"deciduous, single stem","fruits & nuts, forest/wood",...,seeds,"steppe or semiarid (Bs), subtropical humid (Cf...",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Car...,Pecan,"BRIEF DESCRIPTION A tall, handsome, deciduous...",Carya illinoensis


In [57]:
crops[crops['Scientific_Name'].str.contains('ssp.')]
# crops['Variety'].unique()

,Crop_Code,Scientific_Name,Genus,Species,Variety,Life_Form,Habit,Life_Span,Physiology,Category,...,Use_Part,Climate_Zone_Trewartha,USDA_Hardiness_Zone,USDA_Hardiness_Zone_Min,USDA_Hardiness_Zone_Max,Datasheet_URL,PFAF_URL,Common_Name,Notes,Normalized_Scientific_Name
165,921,Digitaria eriantha ssp. eriantha,Digitaria,eriantha,,grass,prostrate/procumbent/semi-erect,perennial,multi stem,"forage/pasture, environmental, weed",...,entire plant,"tropical wet & dry (Aw), tropical wet (Ar), st...",7-10,7,10,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Dig...,crab grass,BRIEF DESCRIPTION A stoloniferous grass up to...,Digitaria eriantha
346,1877,Rubus idaeus ssp. idaeus,Rubus,idaeus,,sub-shrub,erect,perennial,"deciduous, multi stem","fruits & nuts, medicinals & aromatic",...,fruits,"steppe or semiarid (Bs), temperate oceanic (Do...",4-8,4,8,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Rub...,"European raspberry, red garden raspberry, red ...",SOURCES\nDube P 1982 pp 18\nDuke J 1975 pp 26 ...,Rubus idaeus
551,2637,Acacia julifera ssp. gilbertensis,Acacia,julifera,,"shrub, tree",erect,perennial,"single stem, multi stem","ornamentals/turf, forest/wood, environmental",...,bark,"tropical wet & dry (Aw), steppe or semiarid (Bs)",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Aca...,,"BRIEF DESCRIPTION A small, straggly shrub or ...",Acacia julifera
552,2638,Acacia julifera ssp. julifera,Acacia,julifera,,"shrub, tree",erect,perennial,"deciduous, single stem, multi stem","ornamentals/turf, forest/wood, environmental",...,bark,"steppe or semiarid (Bs), subtropical humid (Cf)",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Aca...,,"BRIEF DESCRIPTION A small, straggly shrub or ...",Acacia julifera
610,2763,Achillea millefolium ssp. millefolium,Achillea,millefolium,,herb,erect,perennial,multi stem,medicinals & aromatic,...,flowers,"subtropical dry summer (Cs), temperate oceanic...",4-8,4,8,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Ach...,,,Achillea millefolium
886,3989,Butyrospermum paradoxum ssp. parkii,Butyrospermum,paradoxum,,tree,erect,perennial,"deciduous, single stem","materials, medicinals & aromatic, environmental",...,seeds,"tropical wet & dry (Aw), steppe or semiarid (Bs)",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=But...,,BRIEF DESCRIPTION A small deciduous tree usua...,Butyrospermum paradoxum
902,4058,Callitris preissii ssp. preissii,Callitris,preissii,,"shrub, tree",erect,perennial,"evergreen, single stem, multi stem","ornamentals/turf, forest/wood",...,bark,"subtropical humid (Cf), subtropical dry summer...",8-11,8,11,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Cal...,,BRIEF DESCRIPTION A shrub to medium sized tre...,Callitris preissii
915,4127,Cannabis sativa ssp. indica,Cannabis,sativa,,herb,erect,annual,single stem,materials,...,bark,"tropical wet & dry (Aw), tropical wet (Ar), st...",8-11,8,11,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Can...,"hemp, chanvre, hanf, hennep, canamo, canapa, h...","BRIEF DESCRIPTION A robust, tall, erect herb ...",Cannabis sativa
990,4481,Chenopodium album ssp. album,Chenopodium,album,,herb,erect,annual,single stem,"forage/pasture, vegetables, materials",...,seeds,"steppe or semiarid (Bs), subtropical dry summe...",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Che...,"goosefoot, misbredie, withondebossie",BRIEF DESCRIPTION A herb. USES Cultivated fo...,Chenopodium album
1100,5185,Dactylis glomerata ssp. hispanica,Dactylis,glomerata,,grass,erect,perennial,multi stem,forage/pasture,...,entire plant,"steppe or semiarid (Bs), subtr

In [58]:
list(empty_lists_df[empty_lists_df['Crop_Name']=='Other Small Grains']['Discussion'])

[]

In [59]:
list(empty_lists_df[empty_lists_df['Crop_Name']=='Other Hay/Non Alfalfa']['Discussion'])

[]

In [60]:
list(empty_lists_df[empty_lists_df['Crop_Name']=='Strawberries']['Discussion'])

["When a field has 'strawberries' growing in it, the species is most likely Fragaria × ananassa. Strawberries are cultivated for their sweet, juicy fruits, enjoyed fresh or used in various culinary dishes and desserts."]